<a href="https://colab.research.google.com/github/jimsteeg/jukebox/blob/master/OpenAI_Jukebox_Jimney_Fork_v1_1_RC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **COMPOSER**

In [ ]:
#@title **INITIAL SETUP** - Run once each session

# Set low defaults for unknown GPU. Users can change to match GPU Memory capacity
# Upsampling (level_1 & level_0) require much less GPU memory than level_2 sampling
UPPER_LEVEL_MAX_BATCH_SIZE = 1
LOWER_LEVEL_MAX_BATCH_SIZE = 2

gpu_info = !nvidia-smi -L

# Supported values for this code and these cards. Increasing will probably give OOM at some point
if gpu_info[0].find('Tesla T4') >= 0:
    UPPER_LEVEL_MAX_BATCH_SIZE = 2
    LOWER_LEVEL_MAX_BATCH_SIZE = 8

if gpu_info[0].find('P100') >= 0:
    UPPER_LEVEL_MAX_BATCH_SIZE = 3
    LOWER_LEVEL_MAX_BATCH_SIZE = 9

if gpu_info[0].find('V100') >= 0:
    UPPER_LEVEL_MAX_BATCH_SIZE = 3
    LOWER_LEVEL_MAX_BATCH_SIZE = 9

if gpu_info[0].find('A100') >= 0:
    UPPER_LEVEL_MAX_BATCH_SIZE = 6
    LOWER_LEVEL_MAX_BATCH_SIZE = 12
    # Compatibility install
    !pip install torch==1.9.1+cu111 torchvision==0.10.1+cu111 torchaudio==0.9.1 -f https://download.pytorch.org/whl/torch_stable.html

print(f'This runtime has {gpu_info[0]}\n')

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1024**3

print(f'This runtime has {ram_gb:.1f} GB of RAM\n')
if ram_gb < 25:
    print('Not enough RAM to load 5b_lyrics model\n')

print(f'Setting UPPER_LEVEL_MAX_BATCH_SIZE = {UPPER_LEVEL_MAX_BATCH_SIZE}') #Level_2
print(f'Setting LOWER_LEVEL_MAX_BATCH_SIZE = {LOWER_LEVEL_MAX_BATCH_SIZE}\n') #Level_1, Level 0
print(f'This means you will be able to upsample a maximum of {LOWER_LEVEL_MAX_BATCH_SIZE} samples at a time\n')
print(f'Top level sampling will be broken up into batches of {UPPER_LEVEL_MAX_BATCH_SIZE} samples at a time\n')

# Mount Google Drive
from google.colab import drive

# TODO: Make this optional
#SAVE_MODELS_ON_GDRIVE = True #@param {type:"boolean"}

#@markdown ---

ROOT_DIR = 'gdrive'  #@param {type:"string"}
PROJECT = 'jukebox' #@param {type:"string"}

#@markdown ---

drive.mount(ROOT_DIR)

WORK_DIR    = f'{ROOT_DIR}/MyDrive'
PROJECT_DIR = f'{WORK_DIR}/{PROJECT}/compose'
PROMPTS_DIR = f'{PROJECT_DIR}/prompts'

print('\nInstalling jukebox components. You can ignore the pip resolver error message on Colab. They are not relevant here')
!pip install git+https://github.com/openai/jukebox -q
!pip install av -q
print('')

import jukebox
import datetime
import gc
import glob
import json
import librosa
import os

import numpy as np
import psutil
import soundfile
import torch as t

from datetime import datetime
from glob import glob
from IPython.display import Audio
from pathlib import Path
from PIL import Image, ImageFilter
from pynvml import nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo, nvmlInit

from termcolor import colored

import jukebox.utils.dist_adapter as dist

from jukebox.align import get_alignment
from jukebox.data.labels import EmptyLabeller
from jukebox.hparams import Hyperparams, setup_hparams
from jukebox.lyricdict import gpt_2_lyrics, poems 

from jukebox.save_html import save_html
from jukebox.utils.audio_utils import load_audio, save_wav
from jukebox.utils.dist_utils import print_once, setup_dist_from_mpi
from jukebox.utils.sample_utils import get_starts, split_batch
from jukebox.utils.torch_utils import empty_cache

# MPI Connect. MPI doesn't like being initialized twice, hence the following
# Catching NameError is generally frowned upon but our options are limited and our use is specific
try:
    if device is not None:
        pass
except NameError:
    rank, local_rank, device = setup_dist_from_mpi()    

# Set these to None because later we may cache
zs          = None 
vqvae       = None
top_prior   = None
upsampler_0 = None
upsampler_1 = None

# Use half-point precision for parameters. Usually makes things go vroom. 
# This could be a problem if we ever try to run on TPU.
FP16 = True

# Zag's variable. Keep this for the custom sampling code 
newtosample = True

# Needs a browser capable of displaying or local web server
OUTPUT_HTML = False

# Quick fix to get demo prompt files. Temporary

!wget -ncv https://github.com/jimsteeg/jukebox/raw/jimsteeg-patch-1/jukebox/db42.wav
!wget -ncv https://github.com/jimsteeg/jukebox/raw/jimsteeg-patch-1/jukebox/led.wav
!wget -ncv https://github.com/jimsteeg/jukebox/raw/jimsteeg-patch-1/jukebox/treads.wav
!wget -ncv https://github.com/jimsteeg/jukebox/raw/jimsteeg-patch-1/jukebox/yes.wav

!cp *.wav $PROMPTS_DIR

# Download and cache the models in GDrive (only need to do this once)
# Then copy the models to session cache 
# This will speed up the loading process later when we load these models into memory

dir_cache_models = '/root/.cache/jukebox/models'

dir_cache_5b       = f'{dir_cache_models}/5b'
dir_cache_5blyrics = f'{dir_cache_models}/5b_lyrics'
dir_cache_1blyrics = f'{dir_cache_models}/1b_lyrics'

dir_local_models   = f'{WORK_DIR}/jukebox/models'

dir_local_5b       = f'{dir_local_models}/5b'
dir_local_5blyrics = f'{dir_local_models}/5b_lyrics'
dir_local_1blyrics = f'{dir_local_models}/1b_lyrics'

dir_local_models_glob = f'{dir_local_models}/*' 

if not os.path.exists(dir_local_models):
    print(f'Downloading >10GB of model files from https://openaipublic.azureedge.net/jukebox/models/ into {dir_local_models}\nThis will take a while. If you keep these model files this will only need to be done once\n...')

    !mkdir -pv $dir_local_5b
    !mkdir -pv $dir_local_5blyrics
    !mkdir -pv $dir_local_1blyrics

    file_level_2_1b = f'{dir_local_models}/1b_lyrics/prior_level_2.pth.tar'
    file_level_2_5b = f'{dir_local_models}/5b_lyrics/prior_level_2.pth.tar'
    file_level_1    = f'{dir_local_models}/5b/prior_level_1.pth.tar'       
    file_level_0    = f'{dir_local_models}/5b/prior_level_0.pth.tar'
    file_vqvae      = f'{dir_local_models}/5b/vqvae.pth.tar'                

   #!wget -O $file_level_2_1b https://openaipublic.azureedge.net/jukebox/models/1b_lyrics/prior_level_2.pth.tar
    !wget -O $file_level_2_5b https://openaipublic.azureedge.net/jukebox/models/5b_lyrics/prior_level_2.pth.tar
    !wget -O $file_level_1    https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_1.pth.tar
    !wget -O $file_level_0    https://openaipublic.azureedge.net/jukebox/models/5b/prior_level_0.pth.tar
    !wget -O $file_vqvae      https://openaipublic.azureedge.net/jukebox/models/5b/vqvae.pth.tar

if not os.path.exists(dir_cache_models):
    print(f'Copying >10GB of model files from {dir_local_models_glob} into {dir_cache_models}\nThis will take a while ...')
    
    !mkdir -pv $dir_cache_5b
    !mkdir -pv $dir_cache_5blyrics
    !mkdir -pv $dir_cache_1blyrics

    !cp -rv $dir_local_models_glob $dir_cache_models

#########################################################################################
# FUNCTIONS
#########################################################################################

def MD_LINE():
    print('='*120)

def MD_HEAD(msg=''):
    MD_LINE()
    print(str(msg))
    MD_LINE()

def show_mem(msg=''):
    mem = psutil.virtual_memory()
    GB = 1024**3
    
    total     = round(mem[0]/GB, 2)
    available = round(mem[1]/GB, 2)
    percent   = round(mem[2],    2) 
    used      = round(mem[3]/GB, 2)
    free      = round(mem[4]/GB, 2)

    print(f'### {str(msg)} ### :RAM: {available:.4} GB free')
    
def vram(msg=''):
    # Quick and dirty. More elegance next version
    GB = 1024**3
    mem_reserved = t.cuda.memory_reserved()/GB
    
    free_vram = 0.0

    nvmlInit()
    h = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(h)

    free_vram = round(info.free/GB, 3)
  
    free_vram_msg = colored(free_vram, 'green') 

    if free_vram < 5.0:
        free_vram_msg = colored(free_vram, 'yellow')
    if free_vram < 1.0:
        free_vram_msg = colored(free_vram, 'red')
    
    print(f'GPU VRAM free: ' + free_vram_msg + f' GB: ### {str(msg)} ###')

    empty_cache()

#########################################################################################

def seconds_to_tokens(sec, sr, prior, chunk_size):
    tokens = sec*hps.sr // prior.raw_to_tokens
    tokens = ((tokens // chunk_size) + 1) * chunk_size
    assert tokens <= prior.n_ctx, 'Choose a shorter generation length to stay within the top prior context'
    return tokens      

def dump_song_info(file_path, song, sort_keys=False):
    with open(file_path, 'w') as f:
        json.dump(song, f, sort_keys=sort_keys)

def read_song_info(file_path):
    with open(file_path, 'r') as f:
        song_info = json.load(f)
    return song_info

def update_song_length(song_info_path, song, extend_length_in_seconds): 
    pass
    return     

def fix_encoding(zs):
    prompt_length = (int(sample_hps.prompt_length_in_seconds * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens

    zs2 = primed_sample_top(labels, sampling_kwargs, priors, hps, sample_hps)

    zs[0] = zs2[0]
    zs[1] = zs2[1]

    level_dir = f'{SONG_DIR}/level_2'
    t.save(dict(zs=zs, labels=labels, sampling_kwargs=sampling_kwargs, x=None), f'{level_dir}/data2.pth.tar')  
    
    return zs

def clean_encoding(zs):
    zs[0] = t.zeros(hps.n_samples, 0, dtype=t.long, device='cpu')
    zs[1] = t.zeros(hps.n_samples, 0, dtype=t.long, device='cpu')

    return zs       

#########################################################################################

# Load codes from previous sampling run
def load_codes(codes_file, duration, priors, hps):
    v = hps.v

    device='cpu'

    data = t.load(codes_file, map_location=device)
       
    zs = data['zs']

    # Removing this check because we are going to actually set n_samples based on the number we find in the codes file
    # assert zs[-1].shape[0] == hps.n_samples, f"Expected bs = {hps.n_samples}, got {zs[-1].shape[0]}"

    if v: print(f'loaded codes - zs: {[z.shape for z in zs]}')
    
    labels = data['labels']
    
    sampling_kwargs = data['sampling_kwargs']

    del data 
    
    # This needs a prior to work, but we've moved things around so this needs to be made whole at a later date
#    if duration is not None: 
#        # Cut off codes to match duration
#        top_raw_to_tokens = priors[-1].raw_to_tokens
#        assert duration % top_raw_to_tokens == 0, f"Cut-off duration {duration} not an exact multiple of top_raw_to_tokens"
#        assert duration//top_raw_to_tokens <= zs[-1].shape[1], f"Cut-off tokens {duration//priors[-1].raw_to_tokens} longer than tokens {zs[-1].shape[1]} in saved codes"
#        zs = [z[:,:duration//prior.raw_to_tokens] for z, prior in zip(zs, priors)]

    return zs, labels, sampling_kwargs

# Load `duration` seconds of the given audio files to use as prompts
def load_prompts(audio_files, duration, offset, hps):
    v = hps.v
    
    xs = []
    for audio_file in audio_files:
        x = load_audio(audio_file, sr=hps.sr, duration=duration, offset=0.0, mono=True)
        x = x.T # CT -> TC
        xs.append(x)
    while len(xs) < hps.n_samples:
        xs.extend(xs)
    xs = xs[:hps.n_samples]
    x = t.stack([t.from_numpy(x).to('cpu') for x in xs])

    if v: print(f'x: {x.shape}')

    del xs 

    return x

#########################################################################################
# Modified from Sample.py. Includes checkpointing
#########################################################################################

# Sample a partial window of length<n_ctx with tokens_to_sample new tokens on level=level
def sample_partial_window(zs, labels, sampling_kwargs, level, prior, tokens_to_sample, hps):
    n_ctx = prior.n_ctx
    current_tokens = zs[level].shape[1]
    if current_tokens < n_ctx - tokens_to_sample:
        sampling_kwargs['sample_tokens'] = current_tokens + tokens_to_sample
        start = 0
    else:
        sampling_kwargs['sample_tokens'] = n_ctx
        start = current_tokens - n_ctx + tokens_to_sample
    
    return sample_single_window(zs, labels, sampling_kwargs, level, prior, start, hps)

#########################################################################################

# Sample a single window of length=n_ctx at position=start on level=level
def sample_single_window(zs, labels, sampling_kwargs, level, prior, start, hps):
    v = hps.v
    n_samples = hps.n_samples
    n_ctx = prior.n_ctx
    end = start + n_ctx

    vram('sample_single_window: a1')

    zs = [z.to('cpu') for z in zs]

    if v: print(f'zs in sample_single_window {[z.shape for z in zs]}')

    # get z already sampled at current level
    z = zs[level][:, start:end]
 
    if 'sample_tokens' in sampling_kwargs:
        # Support sampling a window shorter than n_ctx
        sample_tokens = sampling_kwargs['sample_tokens']
    else:
        sample_tokens = (end - start)

    conditioning_tokens, new_tokens = z.shape[1], sample_tokens - z.shape[1]

    print(f"Sampling {sample_tokens} tokens for [{start}, {start + sample_tokens}]. Conditioning on {conditioning_tokens} tokens")

    global newtosample
    newtosample = (new_tokens > 0)
    if new_tokens <= 0:
        # Nothing new to sample
        return zs
 
    # get z_conds from level above
    z_conds = prior.get_z_conds(zs, start, end)
 
    # set y offset, sample_length and lyrics tokens
    prior = prior.to('cuda')
    
    vram('sample_single_window: c2')

    y = prior.get_y(labels, start)
  
    max_batch_size = sampling_kwargs['max_batch_size']; del sampling_kwargs['max_batch_size']

    z_list = split_batch(z, n_samples, max_batch_size)
    z_conds_list = split_batch(z_conds, n_samples, max_batch_size)
    y_list = split_batch(y, n_samples, max_batch_size)
    
    if v:
        print(f'y_list: {[y.shape for y in y_list]}')
        print(f'z_list: {[z.shape for z in z_list]}')
        if level < 2:
            if z_conds_list is not None: #Gives NoneType object in top level since there's no z_conds yet
                print(f'z_conds_list: {[z_cond[0].shape for z_cond in z_conds_list]}') 

    z_samples = []
    n_batches = len(y_list)

    for z_i, z_conds_i, y_i, i in zip(z_list, z_conds_list, y_list, range(n_batches)):

        print(f'Sampling batch {i + 1} of {n_batches}')
        
        z_i = z_i.to('cuda')
        
        if z_conds_i is not None:
            z_conds_i = [z.to('cuda') for z in z_conds_i]

        y_i = y_i.to('cuda')
        
        z_samples_i = prior.to('cuda').sample(n_samples=z_i.shape[0], z=z_i, z_conds=z_conds_i, y=y_i, **sampling_kwargs)
        
        vram('sample_single_window: e3')
        prior = prior.to('cpu');  
        vram('sample_single_window: e4')

        if v: print(f'z_samples_i: {z_samples_i.shape}')

        z_samples.append(z_samples_i)   

        del z_samples_i; del z_i; del y_i 
        if z_conds_i is not None: 
            del z_conds_i;
        vram('sample_single_window: e5')  

    if v: print(f'z_samples: length {len(z_samples)}')

    z = t.cat(z_samples, dim=0)

    if v: print(f'z after t.cat(z_samples): {z.shape}')

    sampling_kwargs['max_batch_size'] = max_batch_size

    # Update z with new sample
    z_new = z[:,-new_tokens:]
    if v: print(f'z_new: {z_new.shape}')

    if v: print(f'zs[{level}] before: {zs[level].shape}')
    
    zs[level] = t.cat([zs[level], z_new.to('cpu')], dim=1)
    
    if v: print(f'zs[{level}] after:  {zs[level].shape}')
    
    del z; del z_new; del z_samples; del prior; del y

    if v: print(f'zs from sample_single_window: {[z.shape for z in zs]}')    
    
    return zs

#########################################################################################

# Sample total_length tokens at level=level with hop_length=hop_length
def sample_level(zs, labels, sampling_kwargs, level, prior, total_length, hop_length, hps):

    print(colored(f"\nUPSAMPLING LEVEL {level}:\n", 'yellow'))

    if total_length >= prior.n_ctx:
        starts = get_starts(total_length, prior.n_ctx, hop_length)
        n_steps = 0
        x = None
        for start in starts:
            n_steps += 1
            datea = datetime.now()	

            zs = sample_single_window(zs, labels, sampling_kwargs, level, prior, start, hps)	

            if newtosample and n_steps < len(starts):
                do_output(zs, labels, sampling_kwargs, prior, level, hps)
            
            dateb = datetime.now()
            timex = ((dateb-datea).total_seconds()/60.0) * (len(starts)-n_steps)
            timex = int(timex)

            print(f"\nStep " + colored(n_steps,'yellow') + " of " + colored(len(starts),'yellow') + " complete. Estimated minutes to complete level: " + (colored('???', 'yellow'), colored(timex, 'yellow'))[n_steps > 1 and newtosample])
            print()
    else:
        zs = sample_partial_window(zs, labels, sampling_kwargs, level, prior, total_length, hps)
    
    return zs  

# Sample multiple levels
def _sample(zs, labels, sampling_kwargs, priors, sample_levels, hps):

    v = hps.v
    alignments = None

    for level in reversed(sample_levels):
        prior = priors[level]
        
        total_length = hps.sample_length//prior.raw_to_tokens
        hop_length = int(hps.hop_fraction[level]*prior.n_ctx)
        
        # Set correct total_length, hop_length, labels and sampling_kwargs for level
        assert hps.sample_length % prior.raw_to_tokens == 0, f"Expected sample_length {hps.sample_length} to be multiple of {prior.raw_to_tokens}"

        zs = sample_level(zs, labels[level], sampling_kwargs[level], level, prior, total_length, hop_length, hps)

        # Decode sample
        if v: print(f'zs to decode: {zs[level].shape}')

        do_output(zs, labels, sampling_kwargs, prior, level, hps)

        del prior 
    return zs

#########################################################################################

# Upsample given already generated upper-level codes
def upsample(zs, labels, sampling_kwargs, priors, hps):
    #print('fn upsample')
    sample_levels = list(range(len(priors) - 1))
    zs = _sample(zs, labels, sampling_kwargs, priors, sample_levels, hps)
    return zs    

##########################################################################################
# Pass priors, labels, kwargs as [None, None, top_prior] for compatibility with _sample()
##########################################################################################

# Continue sampling from previously saved codes
def continue_sample_top(zs, labels, sampling_kwargs, priors, hps):
    #print('fn continue_sample_top')
    #sample_levels = [2] 
    #zs = _sample(zs, labels, sampling_kwargs, priors, sample_levels, hps)
    pass #Redo this later
    return zs

# Continue ancestral sampling from previously saved codes
def extend_sample_top(zs, labels, sampling_kwargs, priors, hps, sample_hps):
    #print('fn extend_sample_top')

    prior = priors[-1].to('cpu')

    sample_length_in_tokens = hps.sample_length // prior.raw_to_tokens 

    level = 2  

    if zs is None:
        print('no zs defined yet')
        zs = [t.zeros(hps.n_samples, 0, dtype=t.long, device='cpu') for _ in range(len(priors))]
        sample_hps.mode = 'ancestral'
        print(f'Ancestral sampling ...')

    zs = sample_partial_window(zs, labels[level], sampling_kwargs[level], level, prior, sample_length_in_tokens, hps)

    return zs    

# Prompt the model with raw audio input (dimension: NTC) and generate continuations
def primed_sample_top(labels, sampling_kwargs, priors, hps, sample_hps):
    #print('fn primed_sample_top') 

    sample_hps.mode = 'primed'
    print(f'Primed sampling ...')

    assert sample_hps.audio_file is not None
    assert sample_hps.prompt_length_in_seconds is not None

    audio_files = sample_hps.audio_file.split(',')

    duration = (int(sample_hps.prompt_length_in_seconds * hps.sr) // priors[-1].raw_to_tokens) * priors[-1].raw_to_tokens    
    
    print(f'Encoding audio file: {sample_hps.audio_file}')
    
    offset = 0.0 # Use this later for slicing prompts 
    x = load_prompts(audio_files, duration, offset, hps)

    zs = priors[-1].encode(x, start_level=0, end_level=len(priors), bs_chunks=x.shape[0])

    if VERBOSE:
        print(f'Encoded zs: {[z.shape for z in zs]}')

    del x; x = None

    return zs

#############################################################################################

def get_song_length_in_seconds(level_dir):
    length = 0
    # This is clunky. find a better way
    wav_files = glob(os.path.join(level_dir, '*.wav'))
    if wav_files != []:
        length = librosa.get_duration(filename=wav_files[0]) 

    return length

def save_wav(file_path, x, sr):
    #print('fn save_wav')

    # This should be investigated as an entry point for post-processing the samples
    x = t.clamp(x, -1, 1)
    x = x.numpy()
    for i in list(range(x.shape[0])):
        soundfile.write(f'{file_path}_{i+1:02}.wav', x[i], samplerate=sr, format='wav')    
    del x; x = None

def do_output(zs, labels, sampling_kwargs, prior, level, hps):    
    #print('fn do_output')
    v = hps.v

    zs = [z.to('cpu') for z in zs]; 
    
    prior = prior.to('cpu'); 
        
    # Get the samples for the wav file we're about to make
    x = prior.decode(zs[level:], start_level=level, bs_chunks=zs[level].shape[0])     

    if v: print(f'x after prior.decode: {x.shape}')

    song_dir = hps.name

    if dist.get_world_size() > 1:
        level_dir = f"{song_dir}_rank_{dist.get_rank()}/level_{level}"
    else:
        level_dir = f"{song_dir}/level_{level}"

    if not os.path.exists(level_dir):
        os.makedirs(level_dir)

    song_path  = f'{song_dir}/{song["ALBUM"]}_{song["TITLE"]}_L{level}'
    level_path = f'{level_dir}/{song["ALBUM"]}_{song["TITLE"]}_L{level}'

    save_wav(song_path, x, hps.sr)
    print(f'\nSaved wav files to directory: {song_dir}')   
    save_wav(level_path, x, hps.sr)
    print(f'Saved wav files to directory: {level_dir}') 

    # All the data goes here for safekeeping
    t.save(dict(zs=zs, labels=labels, sampling_kwargs=sampling_kwargs, x=x), f'{level_dir}/data.pth.tar')
    print(f'Saved data file to directory: {level_dir}')   

    if hps.output_html:
        if alignments is None and priors[-1] is not None and priors[-1].n_tokens > 0 and not isinstance(priors[-1].labeller, EmptyLabeller):
            alignments = get_alignment(x, zs, labels[-1], priors[-1], sampling_kwargs[-1]['fp16'], hps)
        save_html(song_dir, x, zs, labels[-1], alignments, hps)
    
    del x; x = None

#############################################################################################

from jukebox.make_models import make_model, make_vqvae 

from jukebox.hparams import Hyperparams, setup_hparams, REMOTE_PREFIX
from jukebox.utils.remote_utils import download
from jukebox.utils.torch_utils import freeze_model
from jukebox.utils.dist_utils import print_all
from jukebox.vqvae.vqvae import calculate_strides

MODELS = {
    '5b':        ("vqvae", "upsampler_level_0", "upsampler_level_1", "prior_5b"),
    '5b_lyrics': ("vqvae", "upsampler_level_0", "upsampler_level_1", "prior_5b_lyrics"),
    '1b_lyrics': ("vqvae", "upsampler_level_0", "upsampler_level_1", "prior_1b_lyrics"),
    #'your_model': ("you_vqvae_here", "your_upsampler_here", ..., "your_top_level_prior_here")
}

def load_checkpoint(path):
    device = 'cpu'
    restore = path
    if restore.startswith(REMOTE_PREFIX):
        remote_path = restore
        local_path = os.path.join(os.path.expanduser("~/.cache"), remote_path[len(REMOTE_PREFIX):])
        local_path = remote_path

        if dist.get_rank() % 8 == 0:
            if not os.path.exists(os.path.dirname(local_path)):
                os.makedirs(os.path.dirname(local_path))
            if not os.path.exists(local_path):
                print("Downloading from azure")
                download(remote_path, local_path)
        restore = local_path
    dist.barrier()
    print(f"Restoring checkpoint from {restore}")
    checkpoint = t.load(restore, map_location=t.device(device))
    print("Checkpoint restored.")
    return checkpoint

def restore_model(hps, model, checkpoint_path):
    model.step = 0
    if checkpoint_path != '':
        checkpoint = load_checkpoint(checkpoint_path)
        # Left from openAI authors
        # checkpoint_hps = Hyperparams(**checkpoint['hps'])
        # for k in set(checkpoint_hps.keys()).union(set(hps.keys())):
        #     if checkpoint_hps.get(k, None) != hps.get(k, None):
        #         print(k, "Checkpoint:", checkpoint_hps.get(k, None), "Ours:", hps.get(k, None))
        checkpoint['model'] = {k[7:] if k[:7] == 'module.' else k: v for k, v in checkpoint['model'].items()}
        model.load_state_dict(checkpoint['model'])
        if 'step' in checkpoint: model.step = checkpoint['step']; print(f'model.step {model.step}')

def make_prior(hps, vqvae, device='cpu'):

    from jukebox.prior.prior import SimplePrior

    prior_kwargs = dict(input_shape=(hps.n_ctx,), bins=vqvae.l_bins,
                        width=hps.prior_width, depth=hps.prior_depth, heads=hps.heads,
                        attn_order=hps.attn_order, blocks=hps.blocks, spread=hps.spread,
                        attn_dropout=hps.attn_dropout, resid_dropout=hps.resid_dropout, emb_dropout=hps.emb_dropout,
                        zero_out=hps.zero_out, res_scale=hps.res_scale, pos_init=hps.pos_init,
                        init_scale=hps.init_scale,
                        m_attn=hps.m_attn, m_mlp=hps.m_mlp,
                        checkpoint_res=hps.c_res if hps.train else 0, checkpoint_attn=hps.c_attn if hps.train else 0, checkpoint_mlp=hps.c_mlp if hps.train else 0)

    x_cond_kwargs = dict(out_width=hps.prior_width, init_scale=hps.init_scale,
                         width=hps.cond_width, depth=hps.cond_depth, m_conv=hps.cond_m_conv,
                         dilation_growth_rate=hps.cond_dilation_growth_rate, dilation_cycle=hps.cond_dilation_cycle,
                         zero_out=hps.cond_zero_out, res_scale=hps.cond_res_scale,
                         checkpoint_res=hps.cond_c_res)  # have to keep this else names wrong

    y_cond_kwargs = dict(out_width=hps.prior_width, init_scale=hps.init_scale,
                         y_bins=hps.y_bins, t_bins=hps.t_bins, sr= hps.sr, min_duration=hps.min_duration,
                         max_duration=hps.max_duration, max_bow_genre_size=hps.max_bow_genre_size)

    if hps.use_tokens and not hps.single_enc_dec:
        prime_kwargs = dict(use_tokens=hps.use_tokens, prime_loss_fraction=hps.prime_loss_fraction,
                            n_tokens=hps.n_tokens, bins=hps.n_vocab,
                            width=hps.prime_width, depth=hps.prime_depth, heads=hps.prime_heads,
                            attn_order=hps.prime_attn_order, blocks=hps.prime_blocks, spread=hps.prime_spread,
                            attn_dropout=hps.prime_attn_dropout, resid_dropout=hps.prime_resid_dropout,
                            emb_dropout=hps.prime_emb_dropout,
                            zero_out=hps.prime_zero_out, res_scale=hps.prime_res_scale,
                            pos_init=hps.prime_pos_init, init_scale=hps.prime_init_scale,
                            m_attn=hps.prime_m_attn, m_mlp=hps.prime_m_mlp,
                            checkpoint_res=hps.prime_c_res if hps.train else 0, checkpoint_attn=hps.prime_c_attn if hps.train else 0,
                            checkpoint_mlp=hps.prime_c_mlp if hps.train else 0)
    else:
        prime_kwargs = dict(use_tokens=hps.use_tokens, prime_loss_fraction=hps.prime_loss_fraction,
                            n_tokens=hps.n_tokens, bins=hps.n_vocab)

    # z_shapes for other levels given this level gets n_ctx codes
    rescale = lambda z_shape: (z_shape[0]*hps.n_ctx//vqvae.z_shapes[hps.level][0],)
    z_shapes = [rescale(z_shape) for z_shape in vqvae.z_shapes]

    prior = SimplePrior(z_shapes=z_shapes,
                        l_bins=vqvae.l_bins,
                        encoder=vqvae.encode,
                        decoder=vqvae.decode,
                        level=hps.level,
                        downs_t=vqvae.downs_t,
                        strides_t=vqvae.strides_t,
                        labels=hps.labels,
                        prior_kwargs=prior_kwargs,
                        x_cond_kwargs=x_cond_kwargs,
                        y_cond_kwargs=y_cond_kwargs,
                        prime_kwargs=prime_kwargs,
                        copy_input=hps.copy_input,
                        labels_v3=hps.labels_v3,
                        merged_decoder=hps.merged_decoder,
                        single_enc_dec=hps.single_enc_dec)

    prior.alignment_head = hps.get('alignment_head', None)
    prior.alignment_layer = hps.get('alignment_layer', None)

    if hps.fp16_params:
        print("Converting to fp16 params")

        from jukebox.transformer.ops import _convert_conv_weights_to_fp16, _move_to_gpu_and_convert_conv_weights_to_fp16
        prior.apply(_convert_conv_weights_to_fp16)
        #prior.apply(_move_to_gpu_and_convert_conv_weights_to_fp16)

    prior = prior.to('cpu')

    restore_model(hps, prior, hps.restore_prior)

    if hps.train:
       print_all(f"Loading prior in train mode")
       pass
    else:
        print(f"Loading prior ...")
        prior.eval()
        freeze_model(prior)
        print(f'Prior loaded in eval mode.')
    return prior    

#############################################################################################

print('\nInitialization complete. You will not need to run this cell again for this session.')

In [ ]:
#@title **WRITE LYRICS**
import random

LYRICS_INDEX = 1 #@param {type:"integer"}

RANDOM_AI_LYRICS = True #@param {type:"boolean"}

print('You can let the runtime pick random lyrics created by GPT-2 or you can add your own lyrics here, using the templates below as examples.\n')

LYRICS = {}

LYRICS[0] = """
I See the sky is falling down the sun is burning round and I'm on the edge I'm out here in the light The end is near and I'm standing here alone. The end is near and I'm standing here alone.  I am the land of the free And I am the sky of burning fire I am the bringer of pain And I'm standing here The end is near and I'm standing here alone. The end is near and I'm standing here alone.  I am the land of the free And I am the sky of burning fire I am the bringer of pain And I'm standing here ready to fight fight fight We'll Bring it all down tonight
"""

LYRICS[1] = """
I need a love to share I need a love for life I need a love to share I need a love for life I wanna make sweet love I wanna make sweet love How many times did I, again, tell you, you know I always love to, love to, gotta love to Where you can have your way You know I love to, love to say You gave it all for love What a thing to be loved You gave it all for love What a thing to be loved Ooh, I love you, yes I do Ooh, you love me, yes you do How many times did I, again, tell you, you know I always love to, love to, gotta love to Where you can have your way You know I love to, love to say How many times did I, again, tell you, you know I always love to, love to, gotta love to Where you can have your way You know I love to, love to say
"""

LYRICS[2] = """
Day and night I walk the streets And call your name Hoping to hear your voice again As I wander through the crowd I can't get away From the only love I need I can't get away There's no place for me to hide I've been lonely So lonely day and night I wander through the crowd And call your name I can't get away From the only love I need I can't get away From the only love I need I can't get away
"""

LYRICS[3] = """
I've been walking in the dark With the shadows in the park, and I need you When I'm down and all alone And there's none to call my own Oh how I need you I've been walkin' in the night With a voice, that's not quite right, and I need you When I'm down and all alone And there's none to call my own, oh how I need you There's a force, I realize When the light is in your eyes, how I need you There's a light I'm waitin for It's in your eyes, don't close the door I still need you
"""

LYRICS[4] = """
Dance on the thunder
Sway in the wind
Will you dance with me?

eyes in the sky
feet in the street
don't ask me why
just dance to the beat

Dance on the thunder
Sway in the wind
Will you dance with me?

The shadows move in
Words of lies dance in the dark

Well come on and dance, yeah, in the evening light
Come on now, come on now and dance

Oh some people dance all night
They burn their needles and cards to the moon

Well come on and dance, yeah, in the evening light
Come on now, come on now and dance

Dance on the thunder
Sway on the wind
Will you dance with me?

eyes in the sky
feet in the street
don't ask me why
just dance to the beat
""" 

LYRICS[5] = """
This time its all so real
No second chance to even win this fight

don't leave me here to deal
My tears of pain cry out into the night

Voice of silence deep within myself
Empty lives we live 
we're living in hell

The storm of rage is getting closer
Rest upon the holy ground

I'll find my way and keep my heart turned down, 
The years are long and fire is my crown

Voice of silence deep within myself
Empty lives we live 
we're burning in hell

Empty wives and all the empty money
All the lies we've told
to keep control

I'll find my way and keep my heart turned down, 
The years are long and fire is my crown

Voice of silence deep within myself
Empty lives we live 
we're burning in hell

Empty wives and all the empty money
All the lies we've told
to keep control
"""

LYRICS[6] = """
one two three four five six seven eight nine ten eleven twelve thirteen fourteeen fifteen
sixteen seventeen eighteen nineteen twenty
"""

####################################################################
display(LYRICS)

if RANDOM_AI_LYRICS:
    lyrics_dict = jukebox.lyricdict.gpt_2_lyrics

    keys = [key for key in lyrics_dict]

    key = keys[random.randint(0, len(keys) - 3)] # Dump the last two. They're crap lyrics 

    LYRICS = lyrics_dict[key]
else:
    LYRICS = LYRICS[LYRICS_INDEX]

print(f'\nChosen Lyrics:')
MD_LINE()
print(f'{LYRICS}')
MD_LINE()

In [ ]:
#@title **CONFIGURE**
#@markdown ---

#@markdown ##### GENERAL SETTINGS - For all modes

# Advanced users can add these settings to the interface.
SR = 44100

LEVELS = 3

MODEL = '5b_lyrics'
#MODEL = '1b_lyrics'

# HOP_LEVEL_2 doesn't get used in this implementation, since we are doing custom top-level sampling.
# These are good defaults but you can change them if you like. Fraction of 1.0 means no overlap so there may be "seams" audible in the final product
# 0.875 seems to have slightly less noise than 0.50 but maybe that's my imagination. Investigate 
HOP_LEVEL_0 = 0.875
HOP_LEVEL_1 = 0.750
HOP_LEVEL_2 = None 

HOPS = [HOP_LEVEL_0, HOP_LEVEL_1, HOP_LEVEL_2]

# This is for the metas where the lyrics tokens are created. No one seems to use it but might be useful for lyrics down the road. Investigate.
OFFSET_IN_SECONDS =  0

CHUNK_SIZE = 16 #default=16
LOWER_LEVEL_CHUNK_SIZE = 32 

# This is hardcoded for now. Makes it easier when top_prior.raw_to_tokens not available.
top_prior_raw_to_tokens = 128

#@markdown ---
SAMPLE_MODE = 'extend'  #@param ['extend','upsample']

#@markdown ---
ALBUM  = 'vega' #@param {type:"string"}
TITLE  = '0105' #@param {type:"string"}

#@markdown ---
SELECT_BEST_SAMPLE = False #@param {type: "boolean"} 

CHOICE = 1 #@param 

#@markdown ---

#@markdown ##### TOP-LEVEL SAMPLING SETTINGS - Upsampling mode will ignore these settings
#@markdown ---
NUMBER_OF_SAMPLES = 9 #@param {type:"integer"}

#@markdown ---
GENRE = 'pop' #@param ['electronic dance pop','acoustic','alternative','ambient','americana','art','avant garde','black metal','bluegrass','blues','brazilian','britpop','celtic','chanson','children','christmas','classic rock','classical','comedy','contemporary','country','crossover','dance','dark','death metal','deathcore','deutschrock','disco','doom','dream','ebm','electro','electronic','electronica','electropop','emo','eurodance','europop','experimental','folk','french','funk','gangsta','garage','glam','gospel','gothic','grindcore','grunge','hardcore','heavy metal','hip hop','horrorcore','house','indie','industrial','jazz','latin','lo fi','medieval','melodic','metal','metalcore','movie','musical','new wave','norte','nu metal','pop','power metal','psychedelic','punk','r n b','rap','reggae','reggaeton','rock','rock n roll','rockabilly','salsa','samba','schlager','sertanejo','singer songwriter','ska','soft','soul','soundtrack','southern','swing','symphonic metal','synthpop','tejano','television','thrash','trance','trip hop','unknown','vocal','worship']
 
ARTIST = 'nelly' #@param ['taylor swift','lana del rey','2pac','aerosmith','alice in chains','andrew lloyd webber','belinda carlisle','benny goodman','black sabbath','blood sweat tears','bon jovi','bright eyes','bryan adams','dave clark five','dead can dance','def leppard','dianna krall','disturbed','doris day','duke ellington','ed sheeran','electric light orchestra','elton john','elvis presley','enigma','foo fighters','frank sinatra','george clinton','guns n roses','gwen stefani','hank williams','heart','imagine dragons','ice cube','jim croce','judas priest','justin bieber','kenny g','katy perry','kelly clarkson','kylie minogue','lady gaga','los pericos','madonna','miley cyrus','muse','nat king cole','nelly','nine inch nails','opeth','orbital','otis redding','ozzy osbourne','peggy lee','peter tosh','prince','pyotr ilyich tchaikovsky','queen','ricky skaggs','roy orbison','salt n pepa','santana','seal','staind','styx','tammy wynette','the black eyed peas','the j geils band','the muppets','the notorious b i g','the strokes','the temptations','the weeknd','the who','twenty one pilots','van halen','young jeezy','willie nelson','wiz kalifa'] 

#@markdown ---
# Add your own files here. Make sure they are in PROMPTS_DIR (by default the directory for prompt files is '{WORK_DIR}/jukebox/compose/prompts')
PROMPT_FILE = 'treads.wav' #@param ['None','db42.wav','led.wav','treads.wav','yes.wav']

PROMPT_LENGTH_IN_SECONDS = 9 #@param {type:"number"}

#@markdown ---
TOTAL_LENGTH_IN_SECONDS = 81 #@param {type:"number"}

#@markdown ---
EXTEND_LENGTH_IN_SECONDS = 3 #@param {type: "number"}
EXTEND_NUMBER_OF_TIMES =  5#@param {type: "number"}

#@markdown ---
TEMPERATURE = 0.985 #@param {type:"number"}

TOP_K = 0 
TOP_P = 0.0 

#@markdown ---

#@markdown ##### ALTERNATE DATA SOURCES - Defaults to searching for data.pth.tar in level_0, level_1, then level_2 directories
#@markdown ---
DATA_SOURCE = 'Use Default Data File' #@param ['Use Default Data File','Custom Checkpoint File Path','Custom Data File Path']

DATA_FILE = '' #@param {type:"string"}

CHECKPOINT_FILE = '' #@param {type: "string"}

#@markdown ---

#@markdown ##### MISCELLANEOUS
#@markdown ---
NOTE = '' #@param {type:"string"}

VERBOSE_OUTPUT = False #@param {type: "boolean"}
V = VERBOSE = VERBOSE_OUTPUT

#########################################################################################

#@markdown ---
TOTAL_SAMPLE_LENGTH_IN_SECONDS = TOTAL_LENGTH_IN_SECONDS

SAMPLE_LENGTH_IN_SECONDS = TOTAL_SAMPLE_LENGTH_IN_SECONDS

OFFSET_IN_SECONDS = 0
OFFSET = OFFSET_IN_SECONDS * SR

ALBUM_DIR = f'{PROJECT_DIR}/albums/{ALBUM}'

SONG_DIR  = f'{ALBUM_DIR}/{TITLE}'

if not os.path.exists(PROMPTS_DIR):
    os.makedirs(PROMPTS_DIR)

if not os.path.exists(SONG_DIR):
    os.makedirs(SONG_DIR)    

PROMPT_FILE_PATH = ''
if PROMPT_FILE.upper() != 'NONE':
    PROMPT_FILE_PATH = f'{PROMPTS_DIR}/{PROMPT_FILE}' 

SONG_INFO_PATH   = f'{SONG_DIR}/{ALBUM}_{TITLE}.json'

if os.path.exists(SONG_INFO_PATH):
    print(f'Reading song info from file: {SONG_INFO_PATH}')
    song = read_song_info(SONG_INFO_PATH)
else:
    print(f'No config file found. Using form fields to initialize')
    song = {
        "SONG_INFO_PATH"   : SONG_INFO_PATH,
        "SONG_DIR"         : SONG_DIR,
        "SAMPLE_MODE"      : SAMPLE_MODE, 
        "PROMPT_FILE_PATH" : PROMPT_FILE_PATH,
        "ALBUM"            : ALBUM,
        "ARTIST"           : ARTIST,
        "GENRE"            : GENRE,
        "TITLE"            : TITLE,
        "LYRICS"           : LYRICS,
        "NOTE"             : NOTE,
        "HOPS"             : HOPS,
        "TEMPERATURE"      : TEMPERATURE,

        "PROMPT_LENGTH_IN_SECONDS"       : PROMPT_LENGTH_IN_SECONDS,
        "TOTAL_SAMPLE_LENGTH_IN_SECONDS" : TOTAL_SAMPLE_LENGTH_IN_SECONDS,
        "SAMPLE_LENGTH_IN_SECONDS"       : SAMPLE_LENGTH_IN_SECONDS,
    }
    dump_song_info(SONG_INFO_PATH, song, sort_keys=False)

#########################################################################################
# Hyperparameters, Sample parameters, Metas
#########################################################################################

hps = Hyperparams()

hps.levels      = LEVELS
hps.model       = MODEL
hps.sr          = SR
hps.n_samples   = NUMBER_OF_SAMPLES 
hps.output_html = OUTPUT_HTML
hps.v           = VERBOSE

hps.name                           = song['SONG_DIR']
hps.hop_fraction                   = song['HOPS'] 

hps.sample_length_in_seconds       = song['SAMPLE_LENGTH_IN_SECONDS']
hps.total_sample_length_in_seconds = song['TOTAL_SAMPLE_LENGTH_IN_SECONDS']

hps.total_sample_length            = (int(hps.total_sample_length_in_seconds * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens # make sample_length a multiple of 128 
hps.sample_length                  = (int(hps.sample_length_in_seconds * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens # make sample_length a multiple of 128

###################################################
#hps.labels = False; song['LYRICS'] = '' # A test
###################################################

#########################################################################################

sample_hps = Hyperparams()

sample_hps.chunk_size                 = CHUNK_SIZE 
sample_hps.lower_level_chunk_size     = LOWER_LEVEL_CHUNK_SIZE

sample_hps.max_batch_size             = UPPER_LEVEL_MAX_BATCH_SIZE
sample_hps.lower_batch_size           = LOWER_LEVEL_MAX_BATCH_SIZE
sample_hps.lower_level_max_batch_size = LOWER_LEVEL_MAX_BATCH_SIZE

sample_hps.mode                       = SAMPLE_MODE.lower() # [extend, truncate, upsample]
sample_hps.audio_file                 = song['PROMPT_FILE_PATH'] 
sample_hps.prompt_length_in_seconds   = song['PROMPT_LENGTH_IN_SECONDS']
sample_hps.temperature                = song['TEMPERATURE']

sample_hps.prompt_length = ((sample_hps.prompt_length_in_seconds * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens

#########################################################################################
# Loading checkpoints
#########################################################################################

def load_zs_and_labels(codes_file, duration, hps):
    print(f'Loading from data file: {sample_hps.codes_file}')
    #zs, labels, sampling_kwargs = load_codes(sample_hps.codes_file, duration, priors, hps)
    priors = None #bit of a hack here. fix later
    
    zs, labels, _ = load_codes(sample_hps.codes_file, duration, priors, hps)

    return zs, labels

if DATA_SOURCE.upper()=='USE DEFAULT DATA FILE':
    file_type = 'data'
    # Identify the lowest level generated.
    for level in [0, 1, 2]:
        data_file_path = f'{SONG_DIR}/level_{level}/data.pth.tar'

        if os.path.isfile(data_file_path):
            sample_hps.codes_file = data_file_path
            print(f'Loading from data file: {sample_hps.codes_file}')
            duration = None
            zs, labels = load_zs_and_labels(sample_hps.codes_file, duration, hps)
            #if V: print(f'zs\n{zs}')
            if V: print(f'labels')
            break

if DATA_SOURCE.upper()=='CUSTOM DATA FILE PATH':
    file_type = 'data'
    sample_hps.codes_file = f'{DATA_FILE}' 
    print(f'Using form field to set custom {file_type} file path: {sample_hps.codes_file}')
    
    if os.path.isfile(sample_hps.codes_file):
        duration = None
        zs, labels = load_zs_and_labels(sample_hps.codes_file, duration, hps)
    else:
        print(f'No data file at {sample_hps.codes_file}')

if DATA_SOURCE.upper()=='CUSTOM CHECKPOINT FILE PATH':
    file_type = 'checkpoint'
    sample_hps.ckpt_file = f'{CHECKPOINT_FILE}'
    print(f'Using form field to set custom {file_type} file path: {sample_hps.ckpt_file}')

    if os.path.isfile(sample_hps.ckpt_file):
        print(f'Loading from {file_type} file: {sample_hps.ckpt_file}')
        zs = t.load(f'{sample_hps.ckpt_file}') 
    else:
        print(f'No checkpoint file at {sample_hps.ckpt_file}')

#############################################################################

# Change to primed if we see a prompt file and zs is empty
if PROMPT_FILE.upper() != 'NONE' and zs is None:
    SAMPLE_MODE = 'primed'
    print(f'Using {PROMPT_FILE} as a prompt')
    print(f'Prompt length: {PROMPT_LENGTH_IN_SECONDS} seconds')
sample_hps.mode = SAMPLE_MODE

print(f'\nSAMPLE_MODE: {sample_hps.mode}\n')

#############################################################################

if zs is not None:
    hps.n_samples = NUMBER_OF_SAMPLES = zs[-1].shape[0]
    print(f'Found samples in {file_type} file')
    print(f'NUMBER_OF_SAMPLES: {NUMBER_OF_SAMPLES}\n')

metas = [dict(
                genre  = song['GENRE'],
                artist = song['ARTIST'],
                lyrics = song['LYRICS'],
              
                total_length = hps.total_sample_length,
                offset = OFFSET 
          ),
        ] * hps.n_samples 

MD_HEAD('SONG INFO')
display(song)
MD_LINE()

if V:
    print('SONG METAS')
    MD_LINE()
    display(metas)
    MD_HEAD('CONFIG PARAMETERS')
    display(hps)
    MD_HEAD('SAMPLE PARAMETERS')
    display(sample_hps)
    MD_LINE()

In [ ]:
#@title **CREATE!**

# Maybe change this so to use the sampling_kwargs read from data file. Since this pretty much stays constant for now, we'll just set it each time
# There's a placeholder in load_codes if we decide otherwise
sampling_kwargs = [
                   dict(temp=0.99, fp16=FP16, max_batch_size=sample_hps.lower_level_max_batch_size, chunk_size=sample_hps.lower_level_chunk_size),
                   dict(temp=0.99, fp16=FP16, max_batch_size=sample_hps.lower_level_max_batch_size, chunk_size=sample_hps.lower_level_chunk_size),
                   dict(temp=sample_hps.temperature, fp16=FP16,  
                        max_batch_size=sample_hps.max_batch_size, chunk_size=sample_hps.chunk_size, 
                        top_k=TOP_K, top_p=TOP_P)
                   ]

def show_zs(zs, msg=''):
    MD_HEAD(msg)
    if zs:
        for level, z in enumerate(zs):
            print(f'level_{level}: {z.shape}')
            display(z)
    MD_LINE()

if sample_hps.mode=='upsample':
    #########################################################################################
    # Upsample
    #########################################################################################

    # Redo haircut code. Double check this works properly.
    # update - kludgy but works if level_2 file is of correct length and exists 
    # otherwise use TOTAL_SAMPLE_LENGTH_IN_SECONDS with a possible haircut
    #song['SAMPLE_LENGTH_IN_SECONDS'] = song['TOTAL_SAMPLE_LENGTH_IN_SECONDS']
    
    song_dir = hps.name

    level_dir = f"{song_dir}/level_2"

    length = get_song_length_in_seconds(level_dir)

    song['TOTAL_SAMPLE_LENGTH_IN_SECONDS'] = length 
    
    #song['TOTAL_SAMPLE_LENGTH_IN_SECONDS'] = 150
    
    hps.total_sample_length_in_seconds = song['TOTAL_SAMPLE_LENGTH_IN_SECONDS']
    hps.sample_length_in_seconds       = song['TOTAL_SAMPLE_LENGTH_IN_SECONDS']

    hps.total_sample_length = (int(hps.total_sample_length_in_seconds * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens
    hps.sample_length = (int(hps.sample_length_in_seconds * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens

    metas = [dict(
                    genre  = song['GENRE'],
                    artist = song['ARTIST'],
                    lyrics = song['LYRICS'],

                    total_length = hps.total_sample_length,
                    offset = OFFSET,
            ),
            ] * hps.n_samples

    # we need > 16GB VRAM to keep top_prior on GPU so we drop it here
    if top_prior is not None:
        del top_prior; top_prior = None

    # Use this to reset upsamplers
    #del upsampler_0; upsampler_0 = None
    #del upsampler_1; upsampler_1 = None

    REMOTE_PREFIX = LOCAL_PREFIX = '/root/.cache'

    RESTORE_PRIOR_LEVEL_0 = f'{LOCAL_PREFIX}/jukebox/models/5b/prior_level_0.pth.tar'
    RESTORE_PRIOR_LEVEL_1 = f'{LOCAL_PREFIX}/jukebox/models/5b/prior_level_1.pth.tar'
    RESTORE_PRIOR_LEVEL_2 = f'{LOCAL_PREFIX}/jukebox/models/5b_lyrics/prior_level_2.pth.tar'

    vqvae, *priors = MODELS[hps.model]
    RESTORE_VQVAE = f'{LOCAL_PREFIX}/jukebox/models/5b/vqvae.pth.tar'
    print(f'Making vqvae ... ')
    vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length=hps.total_sample_length, restore_vqvae=RESTORE_VQVAE)), device='cpu')

    print(f'Making upsampler_0 ...')
    if upsampler_0 is None:
        upsampler_0 = make_prior(setup_hparams(priors[0], dict(restore_prior=RESTORE_PRIOR_LEVEL_0)), vqvae, device='cpu')

    print(f'Making upsampler_1 ...')
    if upsampler_1 is None:
        upsampler_1 = make_prior(setup_hparams(priors[1], dict(restore_prior=RESTORE_PRIOR_LEVEL_1)), vqvae, device='cpu')

    upsamplers = [upsampler_0, upsampler_1]

    if type(labels) == dict:
        labels = [prior.labeller.get_batch_labels(metas, device='cpu') for prior in upsamplers] + [labels]
    else:
        labels = [prior.labeller.get_batch_labels(metas, device='cpu') for prior in upsamplers] + [labels[-1]]

    if SELECT_BEST_SAMPLE:
        print(f'Selecting sample {CHOICE} for {sample_hps.mode}')
        zs[2]=zs[2][CHOICE].repeat(hps.n_samples, 1)

    if V: show_zs(zs, 'TOKENS FROM EARLIER SAMPLES')

    upsample(zs, labels, sampling_kwargs, [*upsamplers, top_prior], hps)    

    if V: show_zs(zs, 'TOKENS AFTER UPSAMPLING')

else:
    #########################################################################################
    # Using one of the top-level sampling modes ['primed', 'extend']
    #########################################################################################

    if top_prior is None:
        if upsampler_0 is not None:
            del upsampler_0; upsampler_0 = None
        if upsampler_1 is not None:
            del upsampler_1; upsampler_1 = None
        
        if vqvae is not None:
            del vqvae; vqvae = None

        #'/root/.cache/jukebox/models/5b/vqvae.pth.tar'
        #'/root/.cache/jukebox/models/5b/prior_level_1.pth.tar'
        #'/root/.cache/jukebox/models/5b/prior_level_0.pth.tar' 
        #'/root/.cache/jukebox/models/5b_lyrics/prior_level_2.pth.tar'
        #'/root/.cache/jukebox/models/1b_lyrics/prior_level_2.pth.tar'

        LOCAL_PREFIX = REMOTE_PREFIX = '/root/.cache'

        RESTORE_VQVAE = f'{LOCAL_PREFIX}/jukebox/models/5b/vqvae.pth.tar'

        print(f'Models: {MODELS[hps.model]}')
        
        vqvae, *priors = MODELS[hps.model]
        
        print(f'Making vqvae ... ')
        vqvae = make_vqvae(setup_hparams(vqvae, dict(sample_length=hps.total_sample_length, restore_vqvae=RESTORE_VQVAE)), device='cpu') 
   
        vqvae = vqvae.to('cpu') 

        RESTORE_PRIOR = f'{LOCAL_PREFIX}/jukebox/models/5b_lyrics/prior_level_2.pth.tar'
        #RESTORE_PRIOR = f'{LOCAL_PREFIX}/jukebox/models/1b_lyrics/prior_level_2.pth.tar'

        print(f'Making top-level prior ... ')
        top_prior = make_prior(setup_hparams(priors[-1], dict(restore_prior=RESTORE_PRIOR)), vqvae, device='cpu')
        
        print(f'vqvae.sample_length: {vqvae.sample_length}')
    
    top_labels = top_prior.labeller.get_batch_labels(metas, device='cpu')   

    priors = [None, None, top_prior]
    labels = [None, None, top_labels] 
    
    #########################################################################################
    # Top-level sampling 
    #########################################################################################
   
    if sample_hps.mode in ['primed', 'extend']:

        hps.sample_length_in_seconds = EXTEND_LENGTH_IN_SECONDS
        hps.sample_length_in_tokens = seconds_to_tokens(EXTEND_LENGTH_IN_SECONDS, hps.sr, top_prior, sample_hps.chunk_size)
        hps.sample_length = ((EXTEND_LENGTH_IN_SECONDS * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens

        if sample_hps.mode == 'primed':
            assert sample_hps.prompt_length_in_seconds is not None

            prompt_length = (int(sample_hps.prompt_length_in_seconds * hps.sr) // top_prior_raw_to_tokens) * top_prior_raw_to_tokens

            zs = primed_sample_top(labels, sampling_kwargs, priors, hps, sample_hps)

        if sample_hps.mode == 'extend':
            if SELECT_BEST_SAMPLE:
                print(f'Selecting sample {CHOICE} for {sample_hps.mode}')
                zs[2]=zs[2][CHOICE].repeat(hps.n_samples, 1)

        if V: show_zs(zs, 'TOKENS FROM EARLIER SAMPLES')

        for i in range(1, EXTEND_NUMBER_OF_TIMES + 1):
            print(f'\n{i} of {EXTEND_NUMBER_OF_TIMES}: Extending in {EXTEND_LENGTH_IN_SECONDS} second chunks using top-level prior for sampling\n')

            zs = extend_sample_top(zs, labels, sampling_kwargs, priors, hps, sample_hps)
         
            if V: show_zs(zs, 'TOKENS AFTER EXTENDING')

            do_output(zs, labels, sampling_kwargs, top_prior, 2, hps)

            # Used to calculate this a different way. Needs to updated for this code. Leave it here for now
            #print(f"Sampled {EXTEND_LENGTH_IN_SECONDS} of {song['TOTAL_SAMPLE_LENGTH_IN_SECONDS']} seconds. Remaining time: {song['TOTAL_SAMPLE_LENGTH_IN_SECONDS'] - get_length_in_seconds()}") 
            #update_song_length(SONG_INFO_PATH, song, EXTEND_LENGTH_IN_SECONDS)

#    if sample_hps.mode == 'continue': # Extend to end of sample length. Not currently used. Will implement later
#        zs = continue_sample_top(zs, labels, sampling_kwargs, priors, hps, sample_hps)            

In [ ]:
 #@title **PLAY**

#PLAYBACK_START = 0 #@param {type:"number"}

level =  0#@param {type:"integer"}

#write_audio(xs, start=PLAYBACK_START)
wav_path = f'{song["SONG_DIR"]}'

length = get_song_length_in_seconds(wav_path)
print(f'Length: {length:.3f} seconds\n')

for i in range(1, hps.n_samples+1):
    #display(Audio(f'{song["SONG_DIR"]}/song_{i}.wav'))

    wav_path = f'{song["SONG_DIR"]}/{song["ALBUM"]}_{song["TITLE"]}_L{level}_{i:02}.wav'
    print(wav_path)
     
    display(Audio(wav_path))

In [ ]:
#@title **VISUALIZE**

level = 2 #@param {type:"integer"}

# Rudimentary. This needs work. 

import librosa
import librosa.display
import matplotlib.pyplot as plt

def display_spectrogram(audio_file_path, sr, ctr=1):
    y, sr = librosa.load(audio_file_path)

    # Display a spectrogram
    S = np.abs(librosa.stft(y))
    plt.figure(figsize=(20, 5))
    fig_ax = librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max), y_axis='log', x_axis='time')
    plt.colorbar()
    plt.title(f'Power Spectrogram for Song {ctr}')
    plt.show()

def display_chromagram(audio_file_path, sr, ctr=1):
    y, sr = librosa.load(audio_file_path)

    C = librosa.feature.chroma_cqt(y=y, sr=sr)
    plt.figure(figsize=(20, 5))
    fig_ax = librosa.display.specshow(C, y_axis='chroma')
    plt.colorbar()
    plt.title(f'Chromagram for Song {ctr}')
    plt.show()

def display_components(audio_file_path, sr, ctr=1):
    y, sr = librosa.load(audio_file_path)

    plt.figure(figsize=(20, 5))
    y_harm, y_perc = librosa.effects.hpss(y)
    librosa.display.waveplot(y_harm, sr=sr, alpha=0.25)
    librosa.display.waveplot(y_perc, sr=sr, color='r', alpha=0.5)
    plt.grid()
    plt.title(f'Harmonic & Percussive for Song {ctr}')
    plt.show()

def display_mfcc(audio_file_path, sr, ctr=1):
    y, sr = librosa.load(audio_file_path)

    C = librosa.feature.mfcc(y=y, sr=sr)
    plt.figure(figsize=(20, 5))
    fig_ax = librosa.display.specshow(C, y_axis='mel')
    plt.colorbar()
    plt.title(f'MFCC Spectrogram for Song {ctr}')
    plt.show()    

def display_wave(audio_file_path, sr, ctr=1):
    y, sr = librosa.load(audio_file_path)

    plt.figure(figsize=(20, 5))
    fig_ax = librosa.display.waveplot(y, sr=sr, alpha=0.75)
    plt.grid()
    plt.title(f'Waveform for Song {ctr}')
    plt.show()

###########################################################################

for i in range(1, hps.n_samples + 1):
    wav_path = f'{song["SONG_DIR"]}/{song["ALBUM"]}_{song["TITLE"]}_L0_{i:02}.wav'
    display_wave(wav_path, hps.sr, ctr=i)   
print()

for i in range(1, hps.n_samples + 1):
    wav_path = f'{song["SONG_DIR"]}/{song["ALBUM"]}_{song["TITLE"]}_L0_{i:02}.wav'
    display_components(wav_path, hps.sr, ctr=i)   
print()

for i in range(1, hps.n_samples + 1):
    wav_path = f'{song["SONG_DIR"]}/{song["ALBUM"]}_{song["TITLE"]}_L0_{i:02}.wav'
    display_spectrogram(wav_path, hps.sr, ctr=i)    
print()

# This one doesn't seem very useful here
#for i in range(hps.n_samples):
#    display_mfcc(f'{song["SONG_DIR"]}/item_{i}.wav', hps.sr, ctr=i)   
#print()

# **UTILITIES**

In [ ]:
#@title **LISTS OF GENRES & ARTISTS TO CHOOSE FROM**

# Need a clever way to display/search/choose from 4000+ artists
# Currently just manually adding to the parameter list in the config settings
# This is less than ideal 

GENRE_LIST = ['acoustic','aerosmith','alternative','ambient','american','americana','art','avantgarde','blackmetal','bluegrass','blues','brazilian','britpop','celtic','chanson','children','christmas','classicrock','classical','comedy','contemporary','country','crossover','dance','dark','deathmetal','deathcore','deutschrock','disco','doom','dream','ebm','electro','electronic','electronica','electropop','emo','eurodance','europop','experimental','fado','folk','french','funk','gangsta','garage','glam','gospel','gothic','grindcore','grunge','hardcore','heavymetal','hiphop','horrorcore','house','indie','industrial','jazz','latin','lofi','medieval','melodic','metal','metalcore','movie','musical','neofolk','newwave','norte','numetal','pop','powermetal','psychedelic','punk','rnb','rap','reggae','reggaeton','rock','rocknroll','rockabilly','salsa','samba','schlager','sertanejo','singersongwriter','ska','soft','soul','soundtrack','southern','swing','symphonic metal','synthpop','tejano','television','thrash','trance','trip hop','unknown','vocal','worship']

#['2pac','aerosmith','alice in chains','andrew lloyd webber','belinda carlisle','benny goodman','black sabbath','blood sweat tears','bon jovi','bright eyes','bryan adams','dave clark five','dead can dance','def leppard','dianna krall','disturbed','doris day','duke ellington','ed sheeran','electric light orchestra','elton john','elvis presley','enigma','foo fighters','frank sinatra','george clinton','guns n roses','gwen stefani','hank williams','heart','imagine dragons','ice cube','jim croce','judas priest','justin bieber','kenny g','katy perry','kelly clarkson','kylie minogue','lady gaga','miley cyrus','muse','nat king cole','nelly','nine inch nails','opeth','orbital','otis redding','ozzy osbourne','peggy lee','prince','pyotr_ilyich_tchaikovsky','queen','roy orbison','salt n pepa','santana','seal','staind','styx','tammy wynette','the black eyed peas','the j geils band','the muppets','the notorious b i g','the strokes','the temptations','the weeknd','the who','twenty one pilots','van halen','young jeezy','willie nelson','wiz kalifa'] 

ARTIST_LIST = ['''
unknown;0
andr_s_schiff;1
sonny_terry;2
nelly;3
markus_schulz;4
modest_petrovich_mussorgsky;5
otis_redding;6
aerosmith;7
kenny_g;8
james_taylor;9
bobby_bland;10
burning_spear;11
skip_james;12
heart;13
tammy_wynette;14
muse;15
beres_hammond;16
james_newton_howard;17
nelson_freire;18
benny_goodman;19
hank_williams;20
they_might_be_giants;21
the_brian_jonestown_massacre;22
lady_gaga;23
chris_young;24
alison_krauss_union_station;25
seal;26
the_hollies;27
shabba_ranks;28
paul_young;29
iration;30
buck_owens;31
the_weeknd;32
elton_john;33
smokey_robinson;34
roy_orbison;35
headhunterz;36
blondie;37
the_temptations;38
ray_stevens;39
foo_fighters;40
christoph_eschenbach;41
blind_willie_mctell;42
al_martino;43
edwin_fischer;44
victor_young;45
justin_bieber;46
styx;47
doris_day;48
tex_beneke;49
the_monkees;50
richard_wagner;51
bryan_adams;52
alessandro_scarlatti;53
rebelution;54
pitbull;55
nat_king_cole;56
wiz_khalifa;57
roger_miller;58
andy_williams;59
peggy_lee;60
pyotr_ilyich_tchaikovsky;61
booker_t_the_mg_s;62
cilla_black;63
billy_fury;64
vera_lynn;65
enrico_caruso;66
sly_and_robbie;67
the_pretenders;68
the_sweet;69
kylie_minogue;70
kay_kyser;71
san_francisco_symphony;72
prince;73
queen;74
kool_the_gang;75
horace_andy;76
midnite;77
gentleman;78
wilhelm_kempff;79
busta_rhymes;80
the_pogues;81
def_leppard;82
al_jolson;83
king_tubby;84
hot_chocolate;85
delroy_wilson;86
jody_watley;87
bobby_vee;88
johnny_mathis;89
the_rascals;90
sviatoslav_richter;91
fred_astaire;92
john_holt;93
amy_grant;94
b_b_king;95
paul_weston;96
four_tops;97
jay_sean;98
pat_boone;99
george_frideric_handel;100
bing_crosby;101
shalamar;102
tommy_dorsey;103
ludacris;104
kenny_chesney;105
murray_perahia;106
lightnin_hopkins;107
ricky_nelson;108
clint_mansell;109
tom_petty_and_the_heartbreakers;110
gary_glitter;111
ringo_starr;112
phil_collins;113
leo_reisman;114
al_green;115
jim_reeves;116
chris_brown;117
cliff_edwards;118
buddy_guy;119
angelo_badalamenti;120
frank_sinatra;121
santana;122
the_pussycat_dolls;123
peaches_herb;124
radu_lupu;125
the_whispers;126
eddy_howard;127
memphis_slim;128
henry_mancini;129
giuseppe_verdi;130
the_dream;131
vladimir_sofronitsky;132
u_roy;133
ken_boothe;134
the_kinks;135
howard_shore;136
hardwell;137
lou_reed;138
calvin_harris;139
eddy_chen;140
anne_murray;141
juice_newton;142
bee_gees;143
wilson_pickett;144
alan_jackson;145
shirley_bassey;146
waylon_jennings;147
destiny_s_child;148
cab_calloway;149
johnny_copeland;150
bright_eyes;151
trey_songz;152
neil_sedaka;153
justin_timberlake;154
arthur_grumiaux;155
the_who;156
the_yardbirds;157
big_joe_turner;158
duke_ellington;159
herb_alpert;160
laura_branigan;161
michael_jackson;162
john_denver;163
peter_gordon;164
solomon_cutner;165
steve_miller_band;166
don_williams;167
the_pointer_sisters;168
metallica;169
the_ink_spots;170
kenny_rogers;171
the_game;172
gene_krupa;173
snoop_dogg;174
j_cole;175
taylor_dayne;176
r3hab;177
guy_lombardo_and_his_royal_canadians;178
shakin_stevens;179
tim_mcgraw;180
olivia_newton_john_john_travolta;181
the_replacements;182
beenie_man;183
diplo;184
sammy_kaye;185
don_gibson;186
shaggy;187
nina_simone;188
stone_temple_pilots;189
ne_yo;190
huddie_william_ledbetter;191
jerry_goldsmith;192
the_bellamy_brothers;193
winifred_atwell;194
georg_philipp_telemann;195
timbaland;196
the_5th_dimension;197
dionne_warwick;198
r_kelly;199
enrique_iglesias;200
sting;201
mikey_dread;202
yellowman;203
eddie_kendricks;204
blur;205
twista;206
paul_anka;207
chris_cornell;208
a_ha;209
pet_shop_boys;210
yo_yo_ma;211
tom_jones;212
neil_diamond;213
vic_damone;214
paul_oakenfold;215
jascha_heifetz;216
t_i_;217
dinah_washington;218
vladimir_ashkenazy;219
leif_ove_andsnes;220
johnny_cash;221
basement_jaxx;222
sonic_youth;223
the_isley_brothers;224
jason_aldean;225
henryk_szeryng;226
lloyd_price;227
reba_mcentire;228
bon_jovi;229
bed_ich_smetana;230
donny_osmond;231
chuck_berry;232
the_smashing_pumpkins;233
israel_vibration;234
stan_kenton;235
conway_twitty_loretta_lynn;236
mcfly;237
r_e_m;238
morgan_heritage;239
lil_wayne;240
garnett_silk;241
lee_scratch_perry;242
howlin_wolf;243
jon_secada;244
goo_goo_dolls;245
brian_hyland;246
abc;247
clarence_gatemouth_brown;248
wilson_phillips;249
atlantic_starr;250
david_guetta;251
s_rgio_mendes;252
fr_d_ric_chopin;253
robert_palmer;254
charlie_musselwhite;255
level_42;256
peter_andre;257
the_spinners;258
erasure;259
philippe_entremont;260
leo_jan_ek;261
the_kingston_trio;262
ronnie_milsap;263
pat_benatar;264
robert_casadesus;265
t_bone_walker;266
duran_duran;267
jerry_reed;268
franz_liszt;269
journey;270
steve_angello;271
showaddywaddy;272
tears_for_fears;273
john_williams;274
daniel_m_ller_schott;275
tampa_red;276
tina_turner;277
the_beatles;278
miranda_lambert;279
howard_jones;280
reo_speedwagon;281
lady_antebellum;282
no_doubt;283
status_quo;284
tiffany;285
billy_eckstine;286
danny_elfman;287
jimmy_wakely;288
ike_tina_turner;289
george_gershwin;290
dinah_shore;291
armin_van_buuren;292
keith_sweat;293
beaux_arts_trio;294
arcangelo_corelli;295
air_supply;296
w_w;297
the_mighty_diamonds;298
harry_gregson_williams;299
blind_lemon_jefferson;300
brook_benton;301
sizzla;302
bobby_darin;303
steely_dan;304
the_skatalites;305
dwight_yoakam;306
clara_haskil;307
jah_cure;308
cliff_richard;309
fabolous;310
sonny_boy_williamson_ii;311
tinie_tempah;312
claude_debussy;313
jewel;314
bob_seger;315
otis_rush;316
mikhail_pletnev;317
gene_autry;318
blind_blake;319
ethel_waters;320
jackie_wilson;321
big_mama_thornton;322
mary_wells;323
the_mills_brothers;324
rage_against_the_machine;325
barbra_streisand;326
frank_crumit;327
the_clash;328
dion;329
all_4_one;330
t_i;331
orchestral_manoeuvres_in_the_dark;332
culture;333
josh_turner;334
one_direction;335
the_rolling_stones;336
half_pint;337
the_searchers;338
vangelis;339
jackson_browne;340
the_beach_boys;341
train;342
prince_buster;343
tavares;344
eve;345
bessie_smith;346
trace_adkins;347
bay_city_rollers;348
david_allan_coe;349
rascal_flatts;350
petula_clark;351
10cc;352
50_cent;353
the_oak_ridge_boys;354
barry_manilow;355
truls_m_rk;356
morrissey;357
gerry_the_pacemakers;358
kay_starr;359
alborosie;360
engelbert_humperdinck;361
new_order;362
tony_bennett;363
stereophonics;364
jimmy_reed;365
akon;366
echo_the_bunnymen;367
jamiroquai;368
stevie_ray_vaughan;369
ben_e_king;370
cheap_trick;371
dusty_springfield;372
mel_tillis;373
damian_marley;374
ruth_etting;375
westlife;376
diana_ross;377
the_shirelles;378
frankie_laine;379
sarah_vaughan;380
ray_price;381
gordon_lightfoot;382
eddie_cantor;383
the_byrds;384
gary_numan;385
bonnie_tyler;386
aswad;387
brownie_mcghee;388
joshua_bell;389
manic_street_preachers;390
mr_vegas;391
tanya_tucker;392
marvin_gaye_tammi_terrell;393
the_staple_singers;394
ry_cooder;395
john_mayall;396
martina_mcbride;397
anner_bylsma;398
magic_sam;399
avril_lavigne;400
robert_nighthawk;401
the_coasters;402
ace_of_base;403
joseph_szigeti;404
artie_shaw;405
big_bill_broonzy;406
the_ventures;407
rick_astley;408
les_paul;409
leonid_kogan;410
hall_oates;411
three_dog_night;412
charley_pride;413
paul_mccartney;414
alfred_cortot;415
crystal_gayle;416
taj_mahal;417
mary_j_blige;418
leann_rimes;419
mildred_bailey;420
ll_cool_j;421
lobo;422
blake_shelton;423
matt_haimovitz;424
beastie_boys;425
johannes_brahms;426
martha_and_the_vandellas;427
lou_rawls;428
the_righteous_brothers;429
rosalyn_tureck;430
vince_gill;431
gary_moore;432
bad_company;433
marty_robbins;434
russ_morgan;435
david_cassidy;436
dj_khaled;437
leonard_pennario;438
glenn_miller;439
kings_of_leon;440
afrojack;441
cyndi_lauper;442
trisha_yearwood;443
diamond_rio;444
patty_loveless;445
sugar_minott;446
willie_nelson;447
thomas_newman;448
georgia_gibbs;449
eric_carmen;450
ricky_skaggs;451
earth_wind_fire;452
peter_tosh;453
joe_bonamassa;454
lonnie_johnson;455
missy_elliott;456
nicki_minaj;457
luther_allison;458
grigory_sokolov;459
luke_bryan;460
alanis_morissette;461
tracy_lawrence;462
sonny_james;463
brandy;464
michael_bolton;465
boswell_sisters;466
antonio_vivaldi;467
fritz_kreisler;468
elvis_costello_the_attractions;469
woody_herman;470
korn;471
hans_zimmer;472
the_osmonds;473
electric_light_orchestra;474
t_rex;475
van_cliburn;476
harry_james;477
glee_cast;478
simple_minds;479
abba;480
the_jam;481
tanya_stephens;482
the_statler_brothers;483
craig_david;484
moby;485
xxxtentacion;486
paul_simon;487
magic_slim;488
natasha_bedingfield;489
ennio_morricone;490
carpenters;491
joe_tex;492
cocoa_tea;493
the_glee_cast;494
martha_argerich;495
charlie_rich;496
memphis_minnie;497
sheryl_crow;498
jan_dean;499
nick_cave_and_the_bad_seeds;500
del_shannon;501
lee_greenwood;502
gary_lewis_the_playboys;503
marcelle_meyer;504
count_basie;505
harold_melvin_the_blue_notes;506
the_fray;507
the_prodigy;508
alicia_keys;509
conway_twitty;510
barrington_levy;511
b_o_b;512
tritonal;513
mario_lanza;514
mac_davis;515
billy_murray;516
igor_stravinsky;517
pretenders;518
jordin_sparks;519
alice_in_chains;520
ohio_players;521
rick_springfield;522
jimmie_rodgers;523
buju_banton;524
linda_ronstadt;525
george_benson;526
2pac;527
soulja_boy;528
the_flaming_lips;529
ignacy_jan_paderewski;530
gloria_estefan;531
ariana_grande;532
black_uhuru;533
tony_pastor;534
sublime;535
snow_patrol;536
daft_punk;537
johnny_winter;538
robbie_williams;539
eddie_rabbitt;540
james_cotton;541
brad_paisley;542
manfred_mann;543
bassnectar;544
margaret_whiting;545
sam_cooke;546
robert_cray;547
the_beautiful_south;548
barbara_mandrell;549
dick_haymes;550
creedence_clearwater_revival;551
chic;552
ray_charles;553
carter_family;554
ti_sto;555
survivor;556
c_line_dion;557
sergei_prokofiev;558
b_la_bart_k;559
the_congos;560
yefim_bronfman;561
laidback_luke;562
darius_rucker;563
ray_anthony;564
incubus;565
carole_king;566
james_brown;567
swv;568
bruno_mars;569
aphex_twin;570
nitty_gritty_dirt_band;571
gustav_mahler;572
the_shadows;573
the_moody_blues;574
reverend_gary_davis;575
sia;576
gaetano_donizetti;577
earl_hooker;578
the_commodores;579
maria_jo_o_pires;580
eric_donaldson;581
elmore_james;582
sean_kingston;583
don_carlos;584
linkin_park;585
jay_the_americans;586
grand_funk_railroad;587
jls;588
frankie_valli;589
lefty_frizzell;590
en_vogue;591
the_cure;592
perry_como;593
johnny_mercer;594
stevie_wonder;595
ernest_tubb;596
ramin_djawadi;597
ashanti;598
rosemary_clooney;599
anne_sophie_mutter;600
helen_forrest;601
augustus_pablo;602
the_carpenters;603
claudio_arrau;604
bob_dylan;605
joe_simon;606
culture_club;607
the_ipana_troubadors;608
jennifer_lopez;609
karyn_white;610
joe;611
sarah_mclachlan;612
j_geils_band;613
dean_martin;614
hank_snow;615
clyde_mcphatter;616
tlc;617
beck;618
jimmy_dean;619
roy_acuff;620
outkast;621
freddie_mcgregor;622
gus_arnheim;623
gramatik;624
merle_haggard;625
steve_lawrence;626
ma_rainey;627
jimmy_dorsey;628
johnny_paycheck;629
arthur_rubinstein;630
talking_heads;631
capleton;632
les_brown;633
leonard_bernstein;634
bobby_goldsboro;635
kendrick_lamar;636
lenny_kravitz;637
nat_shilkret;638
toby_keith;639
junior_wells;640
billy_j_kramer_the_dakotas;641
peter_paul_mary;642
armand_van_helden;643
h_sker_d_;644
alabama;645
eminem;646
felix_mendelssohn_bartholdy;647
richard_goode;648
pink_floyd;649
sara_evans;650
lonnie_donegan;651
boney_m;652
deadmau5;653
lee_ann_womack;654
eric_clapton;655
ray_parker_jr;656
etta_james;657
the_white_stripes;658
gary_u_s_bonds;659
glen_gray_and_the_casa_loma_orchestra;660
bonnie_raitt;661
soulja_boy_tell_em;662
bobby_rydell;663
carly_simon;664
koko_taylor;665
gregory_isaacs;666
red_hot_chili_peppers;667
josef_suk;668
clint_black;669
buddy_clark;670
tool;671
pierre_fournier;672
alice_cooper;673
lucky_dube;674
jorge_bolet;675
don_diablo;676
r_l_burnside;677
klaus_badelt;678
dolly_parton;679
james_horner;680
green_day;681
henry_burr;682
the_doors;683
roxette;684
louis_armstrong;685
jerry_butler;686
louis_prima;687
paul_van_dyk;688
dennis_brown;689
toni_braxton;690
jerry_lee_lewis;691
donna_summer;692
percy_faith;693
willie_dixon;694
elvis_costello;695
youri_egorov;696
dmitri_shostakovich;697
webb_pierce;698
monica;699
pierre_laurent_aimard;700
muddy_waters;701
garth_brooks;702
boyz_ii_men;703
kris_kristofferson;704
duane_eddy;705
coolio;706
gidon_kremer;707
eurythmics;708
randy_travis;709
collie_buddz;710
faith_evans;711
matisyahu;712
brooks_dunn;713
lulu;714
fletcher_henderson;715
eek_a_mouse;716
shlomo_mintz;717
ray_noble;718
bill_withers;719
ub40;720
b_j_thomas;721
mariah_carey;722
olivia_newton_john;723
jackson_5;724
michael_rose;725
three_days_grace;726
glen_campbell;727
keith_urban;728
onerepublic;729
ted_weems;730
johnny_desmond;731
billy_preston;732
billy_vaughn;733
otis_spann;734
marion_harris;735
richard_marx;736
frankie_carle;737
xavier_cugat;738
zz_top;739
faz_l_say;740
new_york_philharmonic;741
teresa_brewer;742
pixies;743
david_oistrakh;744
rory_gallagher;745
ted_lewis_his_band;746
patsy_cline;747
tracy_byrd;748
daniel_shafran;749
johnny_horton;750
billy_ocean;751
don_mclean;752
ginuwine;753
kiss;754
kaskade;755
the_hold_steady;756
rod_stewart;757
rudolf_serkin;758
the_wanted;759
herman_s_hermits;760
rusty_draper;761
jay_z;762
mississippi_john_hurt;763
linval_thompson;764
billie_holiday;765
lawrence_welk;766
john_lee_hooker;767
the_offspring;768
ramones;769
ronan_keating;770
sir_clifford_michael_curzon;771
dierks_bentley;772
bush;773
texas;774
gene_austin;775
after_all;776
depeche_mode;777
eddy_arnold;778
ferry_corsten;779
inxs;780
jimmie_lunceford;781
johann_sebastian_bach;782
maxi_priest;783
travis_tritt;784
girls_aloud;785
meat_loaf;786
brenda_lee;787
modest_mouse;788
glenn_gould;789
mc_hammer;790
erik_satie;791
the_andrews_sisters;792
alicia_de_larrocha;793
america;794
charles_harrison;795
georges_cziffra;796
peter_serkin;797
gladys_knight_the_pips;798
anton_n_dvo_k;799
emil_gilels;800
myra_hess;801
portugal_the_man;802
rufus;803
sergei_rachmaninoff;804
andr_previn;805
natalie_cole;806
the_killers;807
helen_reddy;808
billy_idol;809
shawn_mendes;810
john_lennon;811
joan_jett;812
jimmy_cliff;813
taylor_swift;814
shura_cherkassky;815
marvin_gaye;816
the_drifters;817
paul_revere_the_raiders;818
donovan;819
alma_gluck;820
fleetwood_mac;821
jo_stafford;822
samson_fran_ois;823
vaughn_monroe;824
tennessee_ernie_ford;825
whitesnake;826
iron_maiden;827
blind_boy_fuller;828
eric_church;829
atomic_kitten;830
harry_nilsson;831
paul_specht;832
bob_marley;833
les_baxter;834
aretha_franklin;835
jessie_j;836
robert_johnson;837
maroon_5;838
sheena_easton;839
george_strait;840
bruce_springsteen;841
oasis;842
jack_white;843
fatboy_slim;844
the_jesus_and_mary_chain;845
hank_williams_jr;846
bill_haley_his_comets;847
charlie_daniels;848
pearl_jam;849
the_stylistics;850
sean_paul;851
david_essex;852
zino_francescatti;853
bruce_hornsby;854
system_of_a_down;855
scott_joplin;856
benny_benassi;857
mitch_miller;858
gloria_gaynor;859
steve_winwood;860
drake;861
bone_thugs_n_harmony;862
nickelback;863
john_mellencamp;864
gene_pitney;865
chet_atkins;866
mark_ronson;867
kc_the_sunshine_band;868
chingy;869
maurice_ravel;870
infected_mushroom;871
walter_gieseking;872
pavement;873
deniece_williams;874
tommy_james_the_shondells;875
harry_belafonte;876
robert_schuman;877
george_jones;878
whitney_houston;879
the_ames_brothers;880
max_romeo;881
patti_page;882
p_nk;883
vienna_philharmonic;884
violent_femmes;885
sly_the_family_stone;886
johnny_marvin;887
sammy_davis_jr_;888
nirvana;889
the_turtles;890
the_manhattans;891
the_supremes;892
gregg_allman;893
alton_ellis;894
giacomo_puccini;895
artur_schnabel;896
2_unlimited;897
michael_rabin;898
toto;899
lil_kim;900
shinedown;901
teddy_wilson;902
alpha_blondy;903
the_guess_who;904
my_chemical_romance;905
matchbox_twenty;906
wolfgang_gartner;907
ed_sheeran;908
faron_young;909
fats_waller;910
kate_smith;911
loretta_lynn;912
antonio_meneses;913
tony_martin;914
frankie_vaughan;915
the_mamas_the_papas;916
david_bowie;917
erskine_hawkins;918
the_four_lads;919
queens_of_the_stone_age;920
the_human_league;921
mud;922
bread;923
dixie_chicks;924
backstreet_boys;925
connie_francis;926
the_black_keys;927
paul_weller;928
pieter_wispelwey;929
the_bachelors;930
janet_jackson;931
the_association;932
edwin_starr;933
hank_williams_jr_;934
steven_isserlis;935
the_doobie_brothers;936
jimi_hendrix;937
sousa_s_band;938
ricky_martin;939
the_cranberries;940
paul_whiteman;941
the_saturdays;942
keb_mo;943
the_psychedelic_furs;944
boyzone;945
the_chemical_brothers;946
johnny_tillotson;947
joe_cocker;948
babyface;949
wet_wet_wet;950
tom_petty;951
lesley_gore;952
deorro;953
war;954
wolfgang_schneiderhan;955
isaac_hayes;956
beyonc;957
gordon_jenkins;958
flo_rida;959
captain_tennille;960
adolf_busch;961
bobby_bare;962
johnnie_taylor;963
above_beyond;964
robin_thicke;965
ja_rule;966
barry_white;967
florence_the_machine;968
the_jimi_hendrix_experience;969
gabrielle;970
leon_fleisher;971
yehudi_menuhin;972
george_harrison;973
kate_bush;974
u2;975
nancy_sinatra;976
the_everly_brothers;977
peter_green;978
cat_stevens;979
the_new_seekers;980
itzhak_perlman;981
red_nichols_his_five_pennies;982
the_script;983
ijahman_levi;984
rihanna;985
joan_jett_and_the_blackhearts;986
the_dorsey_brothers_orchestra;987
charlie_barnet;988
lmfao;989
112;990
kelis;991
john_mccormack;992
guns_n_roses;993
gordon_macrae;994
buddy_holly;995
franz_schubert;996
tommy_roe;997
garbage;998
bow_wow;999
belle_and_sebastian;1000
paul_tortelier;1001
johnny_nash;1002
lazar_berman;1003
avicii;1004
5_seconds_of_summer;1005
mississippi_fred_mcdowell;1006
ivan_moravec;1007
little_richard;1008
john_ogdon;1009
dr_dre;1010
andy_russell;1011
sex_pistols;1012
puddle_of_mudd;1013
june_carter_cash;1014
freddie_king;1015
dino_ciani;1016
h_sker_d;1017
nicky_romero;1018
shania_twain;1019
major_lazer;1020
usher;1021
madonna;1022
nsync;1023
the_o_jays;1024
faith_hill;1025
terence_trent_d_arby;1026
gene_chandler;1027
lisa_stansfield;1028
pharrell_williams;1029
david_geringas;1030
dr_alimantado;1031
jack_scott;1032
vladimir_horowitz;1033
genesis;1034
jessica_simpson;1035
peter_gabriel;1036
spike_jones_and_his_city_slickers;1037
leonard_rose;1038
richard_strauss;1039
mgmt;1040
blink_182;1041
ella_fitzgerald;1042
carrie_underwood;1043
evgeny_kissin;1044
olly_murs;1045
ky_mani_marley;1046
rose_royce;1047
desmond_dekker;1048
montell_jordan;1049
little_river_band;1050
t_pain;1051
gary_allan;1052
chubby_checker;1053
the_diamonds;1054
creed;1055
louis_jordan;1056
vernon_dalhart;1057
gilbert_o_sullivan;1058
ziggy_marley;1059
irene_cara;1060
katy_perry;1061
all_saints;1062
benjamin_britten;1063
p_m_dawn;1064
iona_brown;1065
the_lettermen;1066
natalie_imbruglia;1067
viktoria_mullova;1068
zac_brown_band;1069
bj_rk;1070
mutabaruka;1071
jermaine_jackson;1072
the_impressions;1073
spandau_ballet;1074
luther_vandross;1075
thompson_twins;1076
travis_scott;1077
roots_radics;1078
madness;1079
skrillex;1080
the_smiths;1081
van_halen;1082
the_four_seasons;1083
j_b_lenoir;1084
johnny_rivers;1085
bob_marley_the_wailers;1086
albert_collins;1087
arctic_monkeys;1088
billy_joel;1089
joe_nichols;1090
chicago;1091
coldplay;1092
blue;1093
papa_roach;1094
r_e_m_;1095
eddie_money;1096
ellie_goulding;1097
gym_class_heroes;1098
the_charlie_daniels_band;1099
bette_midler;1100
hubert_sumlin;1101
phil_harris;1102
arturo_benedetti_michelangeli;1103
steps;1104
simon_and_garfunkel;1105
gyptian;1106
britney_spears;1107
lang_lang;1108
blackstreet;1109
the_dave_clark_five;1110
the_chi_lites;1111
wolfgang_amadeus_mozart;1112
johnnie_ray;1113
elvis_presley;1114
red_norvo;1115
the_velvet_underground;1116
nelly_furtado;1117
connee_boswell;1118
soundgarden;1119
dr_hook_the_medicine_show;1120
led_zeppelin;1121
boney_m_;1122
gregor_piatigorsky;1123
john_anderson;1124
joni_james;1125
the_all_american_rejects;1126
marcia_griffiths;1127
ben_bernie;1128
eric_prydz;1129
the_three_suns;1130
albert_king;1131
jodeci;1132
inner_circle;1133
don_cornell;1134
jeff_healey;1135
brian_mcknight;1136
the_stranglers;1137
adam_faith;1138
francis_poulenc;1139
fall_out_boy;1140
the_jets;1141
groundation;1142
randy_newman;1143
jane_s_addiction;1144
bobby_vinton;1145
guiomar_novaes;1146
bo_diddley;1147
philadelphia_orchestra;1148
garrick_ohlsson;1149
evanescence;1150
wilhelm_backhaus;1151
the_platters;1152
the_seekers;1153
selena_gomez;1154
eddy_grant;1155
jan_garber;1156
gioacchino_rossini;1157
christina_aguilera;1158
stevie_nicks;1159
color_me_badd;1160
sandie_shaw;1161
migos;1162
the_allman_brothers_band;1163
will_young;1164
new_edition;1165
radiohead;1166
orbital;1167
the_judds;1168
dan_fogelberg;1169
pascal_rog_;1170
leona_lewis;1171
charlie_patton;1172
yann_tiersen;1173
the_abyssinians;1174
frank_ifield;1175
john_mayer;1176
roger_wolfe_kahn;1177
the_mcguire_sisters;1178
junior_reid;1179
bukka_white;1180
macy_gray;1181
billy_currington;1182
the_tremeloes;1183
ac_dc;1184
little_walter;1185
gorillaz;1186
the_hilltoppers;1187
gil_shaham;1188
little_mix;1189
supertramp;1190
third_world;1191
david_garrett;1192
the_lovin_spoonful;1193
chris_lake;1194
louis_lortie;1195
shakira;1196
vanessa_williams;1197
the_bangles;1198
mark_chesnutt;1199
keith_whitley;1200
roxy_music;1201
betty_hutton;1202
cheryl_cole;1203
the_notorious_b_i_g_;1204
kesha;1205
boston;1206
gerald_moore;1207
heinrich_schiff;1208
joy_division;1209
kenny_loggins;1210
nilsson;1211
counting_crows;1212
simply_red;1213
aaron_tippin;1214
sweet;1215
sugababes;1216
florida_georgia_line;1217
lonnie_mack;1218
jacob_miller;1219
the_police;1220
the_chordettes;1221
fats_domino;1222
willy_deville;1223
bob_crosby;1224
roy_clark;1225
dinu_lipatti;1226
alison_krauss;1227
frankie_avalon;1228
ella_mae_morse;1229
linton_kwesi_johnson;1230
glenn_frey;1231
rudy_vall_e_his_connecticut_yankees;1232
dj_snake;1233
toots_the_maytals;1234
annie_lennox;1235
paramore;1236
john_browning;1237
sister_sledge;1238
julian_lloyd_webber;1239
kanye_west;1240
christopher_cross;1241
professor_longhair;1242
flume;1243
sophie_tucker;1244
third_eye_blind;1245
beyonc_;1246
weezer;1247
kelly_clarkson;1248
take_that;1249
ludwig_van_beethoven;1250
the_cars;1251
george_michael;1252
chaka_khan;1253
arty;1254
steel_pulse;1255
bunny_wailer;1256
the_troggs;1257
nick_lucas;1258
keb_mo_;1259
rise_against;1260
mickey_gilley;1261
poison;1262
debbie_gibson;1263
kim_wilde;1264
sammy_davis_jr;1265
jonas_brothers;1266
cage_the_elephant;1267
son_house;1268
birth_control;1269
faithless;1270
cher;1271
seether;1272
new_kids_on_the_block;1273
rage;1274
richie_spice;1275
sam_smith;1276
the_marvelettes;1277
the_jackson_5;1278
bobbie_gentry;1279
macklemore_ryan_lewis;1280
s_club_7;1281
billy_jones_ernest_hare;1282
arcana;1283
blasterjaxx;1284
will_smith;1285
ray_miller;1286
sugar_ray;1287
ziggy_marley_the_melody_makers;1288
aaron_copland;1289
diddy;1290
daughtry;1291
beach_house;1292
the_dillinger_escape_plan;1293
lonestar;1294
foreigner;1295
lionel_richie;1296
roberta_flack;1297
the_carter_family;1298
demi_lovato;1299
joseph_haydn;1300
sander_van_doorn;1301
underworld;1302
deborah_cox;1303
the_grass_roots;1304
bananarama;1305
iggy_azalea;1306
3_doors_down;1307
the_partridge_family;1308
lead_belly;1309
johnny_long;1310
jagged_edge;1311
the_derek_trucks_band;1312
eddie_fisher;1313
pablo_casals;1314
the_original_dixieland_jazz_band;1315
miley_cyrus;1316
john_powell;1317
the_black_eyed_peas;1318
edvard_grieg;1319
maurizio_pollini;1320
nathan_milstein;1321
twenty_one_pilots;1322
rudolf_firku_n_;1323
nine_inch_nails;1324
the_four_aces;1325
jimmy_rogers;1326
salt_n_pepa;1327
yellow_claw;1328
the_strokes;1329
bobby_brown;1330
juelz_santana;1331
staind;1332
bj_rn_ulvaeus_benny_andersson;1333
the_j_geils_band;1334
the_muppets;1335
gwen_stefani;1336
fedde_le_grand;1337
imagine_dragons;1338
leo_sayer;1339
the_animals;1340
the_b_52_s;1341
furry_lewis;1342
ciara;1343
larry_clinton;1344
dire_straits;1345
pato_banton_the_reggae_revol;1346
goodie_mob;1347
disclosure;1348
georges_bizet;1349
sonny_terry_brownie_mcghee;1350
jim_croce;1351
nelson;1352
jason_derulo;1353
guy_mitchell;1354
the_fontane_sisters;1355
sonny_boy_williamson_i;1356
mirah;1357
jimmy_rushing;1358
john_michael_montgomery;1359
michael_nesmith;1360
george_clinton;1361
burt_bacharach;1362
v6;1363
slim_thug;1364
belinda_carlisle;1365
philip_glass;1366
slade;1367
pete_townshend;1368
oingo_boingo;1369
andrew_lloyd_webber;1370
the_collectors;1371
boy_george;1372
utada_hikaru;1373
mel_torm;1374
diana_krall;1375
melanie_c;1376
john_hammond;1377
peter_green_splinter_group;1378
trouble;1379
g_unit;1380
ferlin_husky;1381
arcade_fire;1382
latino;1383
krayzie_bone;1384
man;1385
dave_clark_five;1386
the_stone_roses;1387
young_jeezy;1388
blood_sweat_tears;1389
kumikameli;1390
dmx;1391
ice_cube;1392
eagles;1393
jill_scott;1394
xtc;1395
peggy_march;1396
michael_bubl;1397
raimon;1398
two_mix;1399
dulce_pontes;1400
the_unseen;1401
hank_locklin;1402
the_notorious_b_i_g;1403
bumblefoot;1404
the_busters;1405
rick_ross;1406
tegan_and_sara;1407
skeeter_davis;1408
curtis_mayfield;1409
sade;1410
wings;1411
lorrie_morgan;1412
saga;1413
a_r_rahman;1414
martha_wainwright;1415
nas;1416
will_i_am;1417
kirsty_maccoll;1418
angel;1419
dave_davies;1420
iggy_pop;1421
jojo;1422
sammy_hagar;1423
ray_davies;1424
and_one;1425
neil_young;1426
mikael_wiehe;1427
the_cardigans;1428
cage;1429
dottie_west;1430
keri_hilson;1431
johnny_hallyday;1432
bill_nelson;1433
fifteen;1434
deen;1435
bobby_v;1436
lil_yachty;1437
too_hort;1438
bernard_lavilliers;1439
hank_thompson;1440
the_chieftains;1441
daryl_hall;1442
antonio_carlos_jobim;1443
aventura;1444
the_the;1445
van_morrison;1446
wynonna_judd;1447
gomez;1448
charles_aznavour;1449
m83;1450
gnr;1451
all;1452
emmylou_harris;1453
lee_hazlewood;1454
mew;1455
uriah_heep;1456
yoko_ono;1457
abw_rts;1458
john_legend;1459
d12;1460
kitty_wells;1461
timbiriche;1462
shel_silverstein;1463
cam_ron;1464
rosanne_cash;1465
2_chainz;1466
tricky;1467
8ball_mjg;1468
flatt_scruggs;1469
bill_anderson;1470
emil_ana_torrini;1471
ufo;1472
mos_def;1473
danzig;1474
juan_gabriel;1475
common;1476
raekwon;1477
france_gall;1478
nicole_scherzinger;1479
r_yksopp;1480
sammie;1481
lena_horne;1482
david_byrne;1483
paul_williams;1484
josh_groban;1485
the_gathering;1486
frank_boeijen;1487
scooter;1488
steve_wariner;1489
mika;1490
pete_seeger;1491
tex_ritter;1492
warrant;1493
porter_wagoner;1494
field_music;1495
three_6_mafia;1496
jim_jones;1497
daniel_o_donnell;1498
brentalfloss;1499
wyclef_jean;1500
hey;1501
bizzy_bone;1502
the_mccalmans;1503
blues_traveler;1504
massive_attack;1505
woody_guthrie;1506
art_garfunkel;1507
andrea_bocelli;1508
david_crosby;1509
dream;1510
soul_asylum;1511
natalie_merchant;1512
shawn_colvin;1513
jonny_lang;1514
funeral_for_a_friend;1515
boz_scaggs;1516
example;1517
lionel_hampton;1518
the_tubes;1519
marc_anthony;1520
good_riddance;1521
moonlight;1522
marc_almond;1523
rza;1524
die_rzte;1525
rbd;1526
alejandro_fern_ndez;1527
wanda_jackson;1528
lara_fabian;1529
julio_iglesias;1530
jeff_beck;1531
peabo_bryson;1532
no_fun_at_all;1533
prong;1534
canibus;1535
krs_one;1536
u_s_bombs;1537
trust;1538
stonewall_jackson;1539
jos_feliciano;1540
m_a;1541
polysics;1542
n_e_r_d;1543
sesame_street;1544
lio;1545
myl_ne_farmer;1546
iris_dement;1547
lily_allen;1548
spoken;1549
architects;1550
jack_johnson;1551
molly_hatchet;1552
cypress_hill;1553
future;1554
the_nits;1555
per_gessle;1556
live;1557
beau;1558
deana_carter;1559
lil_flip;1560
fran_oise_hardy;1561
billy_ray_cyrus;1562
pepper;1563
run_d_m_c;1564
levon_helm;1565
insane_clown_posse;1566
stars;1567
jean_michel_jarre;1568
thunder;1569
juanes;1570
simple_plan;1571
method_man;1572
smash_mouth;1573
meek_mill;1574
fat_joe;1575
kenny_wayne_shepherd;1576
jhen_aiko;1577
the_manhattan_transfer;1578
joss_stone;1579
cee_lo_green;1580
tyrese;1581
charlotte_martin;1582
rodney_crowell;1583
acappella;1584
die_prinzen;1585
pentatonix;1586
abney_park;1587
smooth_mcgroove;1588
the_magnetic_fields;1589
the_nylons;1590
wise_guys;1591
coil;1592
do_as_infinity;1593
lords_of_acid;1594
the_church;1595
chris_rea;1596
jota_quest;1597
miguel_bos;1598
gang_starr;1599
masta_ace;1600
brand_new;1601
mac_miller;1602
cathedral;1603
corrosion_of_conformity;1604
country_joe_mcdonald;1605
eric_johnson;1606
grateful_dead;1607
janis_joplin;1608
john_miles;1609
king_gizzard_the_lizard_wizard;1610
ted_nugent;1611
combichrist;1612
alkaline_trio;1613
anathema;1614
angus_julia_stone;1615
anna_ternheim;1616
anthony_phillips;1617
steve_hackett;1618
aviators;1619
banda_calypso;1620
blue_stahli;1621
the_boys;1622
capital_inicial;1623
city_and_colour;1624
colin_hay;1625
collective_soul;1626
dashboard_confessional;1627
david_rovics;1628
david_usher;1629
die_toten_hosen;1630
funny_van_dannen;1631
dirty_heads;1632
tech_n9ne;1633
elisa;1634
emmerson_nogueira;1635
engenheiros_do_hawaii;1636
eric_bibb;1637
maria_muldaur;1638
panic_at_the_disco;1639
punchline;1640
godsmack;1641
self;1642
heideroosjes;1643
sinner;1644
heather_nova;1645
hoobastank;1646
quietdrive;1647
hyde;1648
jaguares;1649
bert_jansch;1650
jonatha_brooke;1651
joni_mitchell;1652
the_band;1653
josh_garrels;1654
josh_woodward;1655
william_fitzsimmons;1656
katie_melua;1657
jamie_cullum;1658
kristin_hersh;1659
kt_tunstall;1660
legi_o_urbana;1661
the_zombies;1662
francesco_de_gregori;1663
m_ward;1664
beth_orton;1665
magnum;1666
motorpsycho;1667
marillion;1668
jars_of_clay;1669
mason_jennings;1670
matt_nathanson;1671
matthew_good;1672
edguy;1673
gamma_ray;1674
minus_the_bear;1675
mohsen_namjoo;1676
nerina_pallot;1677
never_shout_never;1678
regina_spektor;1679
passenger;1680
paul_kelly;1681
the_style_council;1682
peter_hammill;1683
phantom_planet;1684
phil_keaggy;1685
richard_thompson;1686
said_the_whale;1687
samsas_traum;1688
senses_fail;1689
sevendust;1690
seventh_day_slumber;1691
joan_baez;1692
sister_hazel;1693
slightly_stoopid;1694
sophie_zelmani;1695
suzanne_vega;1696
tatiana;1697
teoman;1698
the_choir;1699
charlotte_church;1700
darlene_zschech;1701
the_front_bottoms;1702
the_maine;1703
the_white_buffalo;1704
little_big_town;1705
threshold;1706
tourniquet;1707
everything_but_the_girl;1708
vertical_horizon;1709
vonda_shepard;1710
warren_zevon;1711
sarah_brightman;1712
blackfoot;1713
black_label_society;1714
z_lia_duncan;1715
2;1716
alejandro_lerner;1717
beth_nielsen_chapman;1718
mercury_rev;1719
brian_wilson;1720
barenaked_ladies;1721
carbon_leaf;1722
celtic_woman;1723
hayley_westenra;1724
crowded_house;1725
delta_goodrem;1726
elbow;1727
resurrection_band;1728
nancy_wilson;1729
janis_ian;1730
jann_arden;1731
jill_sobule;1732
jos_augusto;1733
xuxa;1734
k_d_lang;1735
kim_carnes;1736
los_lobos;1737
mandy_moore;1738
marc_cohn;1739
maureen_mcgovern;1740
melissa_manchester;1741
patti_labelle;1742
helene_fischer;1743
laura_pausini;1744
ivan_lins;1745
thal_a;1746
mike_the_mechanics;1747
paul_carrack;1748
natasha_st_pier;1749
michael_mcdonald;1750
olivia;1751
dizzee_rascal;1752
sam_phillips;1753
serge_gainsbourg;1754
jane_birkin;1755
luis_miguel;1756
sondre_lerche;1757
stan_ridgway;1758
susan_boyle;1759
mike_oldfield;1760
ces_ria_vora;1761
agonoize;1762
funker_vogt;1763
god_module;1764
hocico;1765
nachtmahr;1766
suicide_commando;1767
alejandro_escovedo;1768
southside_johnny_the_asbury_jukes;1769
broken_social_scene;1770
vigilantes_of_love;1771
billy_bragg;1772
wilco;1773
frank_black_and_the_catholics;1774
blue_rodeo;1775
brandi_carlile;1776
patty_griffin;1777
calexico;1778
cass_mccombs;1779
chris_knight;1780
conor_oberst;1781
corb_lund;1782
cowboy_junkies;1783
cracker;1784
cross_canadian_ragweed;1785
dave_alvin;1786
drive_by_truckers;1787
eleni_mandell;1788
lucinda_williams;1789
fred_eaglesmith;1790
dar_williams;1791
the_jayhawks;1792
lana_del_rey;1793
tim_o_brien;1794
hank_williams_iii;1795
james_mcmurtry;1796
joe_henry;1797
john_stewart;1798
josh_ritter;1799
lambchop;1800
nanci_griffith;1801
norma_jean;1802
lyle_lovett;1803
matthew_ryan;1804
my_morning_jacket;1805
neko_case;1806
the_new_pornographers;1807
blue_october;1808
okkervil_river;1809
old_97_s;1810
ray_wylie_hubbard;1811
richmond_fontaine;1812
robert_earl_keen;1813
rocky_votolato;1814
ryan_adams;1815
whiskeytown;1816
son_volt;1817
steve_earle;1818
the_avett_brothers;1819
the_bottle_rockets;1820
the_felice_brothers;1821
arlo_guthrie;1822
the_handsome_family;1823
the_mavericks;1824
ten_years_after;1825
the_walkabouts;1826
todd_snider;1827
vic_chesnutt;1828
iron_wine;1829
wovenhand;1830
x;1831
big_audio_dynamite;1832
globe;1833
carter_the_unstoppable_sex_machine;1834
allison_moorer;1835
front_line_assembly;1836
the_national;1837
the_fall;1838
public_image_ltd;1839
public_enemy;1840
wire;1841
a_tribe_called_quest;1842
de_la_soul;1843
aesop_rock;1844
buck_65;1845
caparezza;1846
childish_gambino;1847
the_roots;1848
colbie_caillat;1849
big_sean;1850
dj_gruff;1851
tyler_the_creator;1852
dokken;1853
fun_lovin_criminals;1854
talib_kweli;1855
jane_air;1856
k_i_z;1857
kid_cudi;1858
jedi_mind_tricks;1859
celph_titled;1860
lupe_fiasco;1861
bun_b;1862
scarface;1863
ghostface_killah;1864
robyn;1865
rehab;1866
swollen_members;1867
styles_p;1868
the_streets;1869
wale;1870
joe_budden;1871
tank;1872
10_years;1873
36_crazyfists;1874
apocalyptica;1875
nina_hagen;1876
anastacia;1877
black_stone_cherry;1878
blindside;1879
breaking_benjamin;1880
bring_me_the_horizon;1881
bullet_for_my_valentine;1882
cave_in;1883
chevelle;1884
d_espairsray;1885
death_angel;1886
deftones;1887
demon_hunter;1888
demon;1889
devin_townsend_project;1890
devin_townsend;1891
doa;1892
dir_en_grey;1893
disturbed;1894
dope;1895
drowning_pool;1896
eighteen_visions;1897
entombed;1898
faith_no_more;1899
fear_factory;1900
fightstar;1901
five_finger_death_punch;1902
finger_eleven;1903
flyleaf;1904
grinspoon;1905
guano_apes;1906
h_blockx;1907
halestorm;1908
hamlet;1909
helmet;1910
bt;1911
ill_ni_o;1912
in_flames;1913
in_this_moment;1914
him;1915
j_b_o;1916
katatonia;1917
killswitch_engage;1918
xzibit;1919
lacuna_coil;1920
phish;1921
limp_bizkit;1922
living_colour;1923
viikate;1924
marilyn_manson;1925
megaherz;1926
falco;1927
melvins;1928
monster_magnet;1929
mushroomhead;1930
nonpoint;1931
soil;1932
otep;1933
p_o_d;1934
powerman_5000;1935
primus;1936
project_86;1937
red;1938
kris_allen;1939
rob_zombie;1940
ozzy_osbourne;1941
rollins_band;1942
saliva;1943
sepultura;1944
shihad;1945
skillet;1946
skindred;1947
slipknot;1948
smile_empty_soul;1949
danielson;1950
soilwork;1951
sonic_syndicate;1952
static_x;1953
stone_sour;1954
taproot;1955
the_notwist;1956
the_word_alive;1957
theory_of_a_deadman;1958
therapy;1959
los_tucanes_de_tijuana;1960
manu_chao;1961
volbeat;1962
zebrahead;1963
hed_p_e;1964
and_you_will_know_us_by_the_trail_of_dead;1965
10_000_maniacs;1966
311;1967
77s;1968
yes;1969
david_lee_roth;1970
hillsong;1971
afi;1972
adam_sandler;1973
afterhours;1974
hawkwind;1975
all_about_eve;1976
all_time_low;1977
allison_crowe;1978
amanda_palmer;1979
american_music_club;1980
amplifier;1981
robert_wyatt;1982
anberlin;1983
andrew_bird;1984
ani_difranco;1985
apoptygma_berzerk;1986
apulanta;1987
arab_strap;1988
joseph_arthur;1989
tom_rosenthal;1990
ash;1991
asian_kung_fu_generation;1992
poets_of_the_fall;1993
babas_nicos;1994
bayside;1995
beatsteaks;1996
ben_folds;1997
ben_folds_five;1998
ben_harper;1999
better_than_ezra;2000
bettie_serveert;2001
big_country;2002
big_head_todd_and_the_monsters;2003
big_sugar;2004
billy_talent;2005
today_is_the_day;2006
red_flag;2007
black_rebel_motorcycle_club;2008
megadeth;2009
blonde_redhead;2010
bob_mould;2011
bodeans;2012
bowling_for_soup;2013
buck_tick;2014
butch_walker;2015
butthole_surfers;2016
caf_tacvba;2017
cake;2018
camper_van_beethoven;2019
carmen_consoli;2020
mario_venuti;2021
franco_battiato;2022
catherine_wheel;2023
catupecu_machu;2024
cem_adrian;2025
john_cale;2026
charlie_brown_jr;2027
nena;2028
chumbawamba;2029
clutch;2030
cl;2031
cmx;2032
coheed_and_cambria;2033
cold_war_kids;2034
travis;2035
coma;2036
concrete_blonde;2037
mint_condition;2038
copeland;2039
crash_test_dummies;2040
joe_jackson;2041
cristian_castro;2042
curve;2043
dada;2044
daniel_amos;2045
daniel_johnston;2046
dave_matthews_band;2047
burning_heads;2048
david_gray;2049
david_sylvian;2050
deacon_blue;2051
deerhoof;2052
del_amitri;2053
dinosaur_jr;2054
dirty_projectors;2055
draco_rosa;2056
duncan_sheik;2057
jeremy_camp;2058
edwyn_collins;2059
eels;2060
nightwish;2061
element_of_crime;2062
embrace;2063
enter_shikari;2064
ulver;2065
everclear;2066
everlast;2067
eyeshine;2068
dio;2069
faust_o;2070
feeder;2071
atmosphere;2072
filter;2073
firewater;2074
fishbone;2075
fountains_of_wayne;2076
four_year_strong;2077
steve_green;2078
fresno;2079
gang_of_four;2080
good_charlotte;2081
blood_on_the_dance_floor;2082
graham_coxon;2083
melissa_etheridge;2084
tony_joe_white;2085
guided_by_voices;2086
robert_pollard;2087
guster;2088
elliott_smith;2089
hedley;2090
hole;2091
hollywood_undead;2092
hot_chip;2093
l_arc_en_ciel;2094
ian_brown;2095
idlewild;2096
jimmy_eat_world;2097
fish;2098
ingrid_michaelson;2099
inme;2100
inspiral_carpets;2101
raf;2102
james;2103
jean_leloup;2104
weird_al_yankovic;2105
jeff_buckley;2106
john_frusciante;2107
dr_john;2108
pj_harvey;2109
jonathan_coulton;2110
juliana_hatfield;2111
julieta_venegas;2112
k_s_choice;2113
kaizers_orchestra;2114
kargo;2115
kasabian;2116
keane;2117
kevin_coyne;2118
kevin_devine;2119
kevin_max;2120
rich_mullins;2121
trooper;2122
suzy_bogguss;2123
kill_hannah;2124
kisp_l_s_a_borz;2125
kult;2126
my_life_with_the_thrill_kill_kult;2127
kutless;2128
la_barranca;2129
la_ley;2130
lao_che;2131
lech_janerka;2132
les_cowboys_fringants;2133
les_fatals_picards;2134
les_rita_mitsouko;2135
sparks;2136
lifehouse;2137
lisa_germano;2138
ed_harcourt;2139
lisa_loeb;2140
liz_phair;2141
local_h;2142
lost_dogs;2143
lostprophets;2144
love_and_rockets;2145
lucybell;2146
lulu_santos;2147
gabriel_o_pensador;2148
adam_lambert;2149
madrugada;2150
mancha_de_rolando;2151
manchester_orchestra;2152
mando_diao;2153
foetus;2154
mark_lanegan;2155
matthew_sweet;2156
max_mo_park;2157
mayday_parade;2158
meat_puppets;2159
men_without_hats;2160
meshell_ndegeocello;2161
midnight_oil;2162
dance_gavin_dance;2163
molotov;2164
ov7;2165
monkey_majik;2166
suede;2167
fernando_ortega;2168
motion_city_soundtrack;2169
mudhoney;2170
mutemath;2171
mercyme;2172
m_o_morta;2173
natalia_lafourcade;2174
natewantstobattle;2175
needtobreathe;2176
split_enz;2177
sum_41;2178
no_te_va_gustar;2179
noir_d_sir;2180
t_tes_raides;2181
o_rappa;2182
o_a_r;2183
ocean_colour_scene;2184
omul_cu_obolani;2185
one_ok_rock;2186
2raumwohnung;2187
our_lady_peace;2188
pain;2189
panda;2190
parokya_ni_edgar;2191
pato_fu;2192
paul_westerberg;2193
pere_ubu;2194
pete_yorn;2195
peter_murphy;2196
placebo;2197
plain_white_t_s;2198
pop_will_eat_itself;2199
porcupine_tree;2200
powderfinger;2201
cat_power;2202
casting_crowns;2203
primal_scream;2204
m_tley_cr_e;2205
the_used;2206
raimundos;2207
mark_knopfler;2208
mark_kozelek;2209
danko_jones;2210
relient_k;2211
raffi;2212
renaud;2213
richard_hawley;2214
rickie_lee_jones;2215
the_shins;2216
rilo_kiley;2217
robyn_hitchcock;2218
mose_allison;2219
roy_harper;2220
rucka_rucka_ali;2221
rx_bandits;2222
saez;2223
samiam;2224
sarah_slean;2225
say_anything;2226
scout_niblett;2227
screaming_females;2228
shannon_wright;2229
silverchair;2230
sin_ad_o_connor;2231
siouxsie_and_the_banshees;2232
sixpence_none_the_richer;2233
skank;2234
skunk_anansie;2235
sleater_kinney;2236
sloan;2237
social_distortion;2238
sophie_hunger;2239
e_40;2240
steve_wynn;2241
subsonica;2242
joe_walsh;2243
super_furry_animals;2244
superchunk;2245
supergrass;2246
swervedriver;2247
switchfoot;2248
dido;2249
takida;2250
taking_back_sunday;2251
teenage_fanclub;2252
w_a_s_p;2253
the_afghan_whigs;2254
the_apples_in_stereo;2255
the_ataris;2256
smoking_popes;2257
the_bluetones;2258
the_breeders;2259
the_cat_empire;2260
the_charlatans_uk;2261
the_clarks;2262
guy_clark;2263
the_comsat_angels;2264
the_connells;2265
the_coral;2266
the_cribs;2267
the_cult;2268
bobby_o;2269
the_mission;2270
blue_yster_cult;2271
the_dandy_warhols;2272
the_dear_hunter;2273
the_decemberists;2274
the_early_november;2275
thievery_corporation;2276
the_fratellis;2277
the_gaslight_anthem;2278
jim_brickman;2279
falling_up;2280
the_hives;2281
the_innocence_mission;2282
the_jazz_butcher;2283
the_jesus_lizard;2284
the_lemonheads;2285
babyshambles;2286
the_living_end;2287
the_matrixx;2288
the_mother_hips;2289
the_mountain_goats;2290
the_muffs;2291
the_pillows;2292
the_posies;2293
the_presidents_of_the_united_states_of_america;2294
the_rasmus;2295
the_raveonettes;2296
the_saints;2297
the_samples;2298
bad_religion;2299
the_smithereens;2300
the_soundtrack_of_our_lives;2301
the_tea_party;2302
mayday;2303
the_triffids;2304
the_vines;2305
the_violet_burning;2306
the_wallflowers;2307
testament;2308
the_divine_comedy;2309
third_day;2310
thrice;2311
tindersticks;2312
tism;2313
tit_s;2314
toad_the_wet_sprocket;2315
tocotronic;2316
tom_mcrae;2317
tori_amos;2318
tracy_chapman;2319
trashcan_sinatras;2320
tre_allegri_ragazzi_morti;2321
tub_ring;2322
unkle;2323
unwritten_law;2324
uverworld;2325
vast;2326
verdena;2327
veruca_salt;2328
face_to_face;2329
virus;2330
voltaire;2331
we_the_kings;2332
the_kooks;2333
lindisfarne;2334
seals_crofts;2335
andy_partridge;2336
xutos_pontap_s;2337
yellowcard;2338
yup;2339
leevi_and_the_leavings;2340
zo;2341
zucchero;2342
z;2343
ebnem_ferah;2344
air;2345
alice;2346
boards_of_canada;2347
brian_eno;2348
burzum;2349
daniel_lanois;2350
enigma;2351
juana_molina;2352
lisa_gerrard;2353
nox_arcana;2354
renard;2355
schiller;2356
sigur_r_s;2357
steven_wilson;2358
swans;2359
wolfgun;2360
xiu_xiu;2361
michael_johnson;2362
montgomery_gentry;2363
the_stanley_brothers;2364
john_waite;2365
shelby_lynne;2366
judy_collins;2367
burl_ives;2368
the_irish_rovers;2369
david_wilcox;2370
devendra_banhart;2371
doc_watson;2372
bill_monroe;2373
michael_martin_murphey;2374
gordon_bok;2375
asleep_at_the_wheel;2376
the_browns;2377
nana_mouskouri;2378
jerry_jeff_walker;2379
steve_goodman;2380
malcolm_holcombe;2381
malvina_reynolds;2382
odetta;2383
tom_paxton;2384
strawbs;2385
phil_ochs;2386
harry_chapin;2387
ramblin_jack_elliott;2388
roger_mcguinn;2389
gene_clark;2390
mat_kearney;2391
the_brothers_four;2392
tom_russell;2393
townes_van_zandt;2394
uncle_dave_macon;2395
delbert_mcclinton;2396
john_hiatt;2397
justin_townes_earle;2398
mark_erelli;2399
over_the_rhine;2400
steve_forbert;2401
manfred_mann_s_earth_band;2402
mot_rhead;2403
rudimentary_peni;2404
illapu;2405
inti_illimani;2406
quilapay_n;2407
v_ctor_jara;2408
skylark;2409
adam_green;2410
cold_chisel;2411
guy_sebastian;2412
jefferson_starship;2413
the_alan_parsons_project;2414
ali_project;2415
modern_talking;2416
animal_collective;2417
banco_del_mutuo_soccorso;2418
ben_lee;2419
bryan_ferry;2420
buffy_sainte_marie;2421
colin_blunstone;2422
cursive;2423
elysian_fields;2424
emerson_lake_palmer;2425
gino_vannelli;2426
g_rard_manset;2427
hot_dad;2428
marina_and_the_diamonds;2429
ismo_alanko;2430
kansas;2431
kari_peitsamo;2432
laibach;2433
laurie_anderson;2434
puhdys;2435
na_o_zumbi;2436
roger_waters;2437
rush;2438
the_walker_brothers;2439
hilltop_hoods;2440
wolfgang_ambros;2441
erste_allgemeine_verunsicherung;2442
jacques_brel;2443
rainhard_fendrich;2444
tom_waits;2445
adrian_belew;2446
anne_clark;2447
can;2448
captain_beefheart_and_the_magic_band;2449
deine_lakaien;2450
devo;2451
einst_rzende_neubauten;2452
frank_zappa;2453
goethes_erben;2454
wishbone_ash;2455
death_cab_for_cutie;2456
antony_and_the_johnsons;2457
jandek;2458
nevermore;2459
king_crimson;2460
king_missile;2461
the_residents;2462
steeleye_span;2463
vampire_rodents;2464
the_walkmen;2465
dog_fashion_disco;2466
freak_kitchen;2467
sigh;2468
children_of_bodom;2469
soft_machine;2470
ara_ketu;2471
asa_de_guia;2472
banda_eva;2473
ivete_sangalo;2474
chiclete_com_banana;2475
daniela_mercury;2476
alejandro_sanz;2477
timbalada;2478
juan_luis_guerra;2479
daddy_yankee;2480
alceu_valen_a;2481
luiz_gonzaga;2482
matia_bazar;2483
axelle_red;2484
barbara;2485
benny_neyman;2486
gigi_d_agostino;2487
jacques_higelin;2488
caetano_veloso;2489
gal_costa;2490
jorge_ben;2491
die_flippers;2492
nicole;2493
angra;2494
reinhard_mey;2495
wolf_biermann;2496
florent_pagny;2497
hannes_wader;2498
tienne_daho;2499
henri_salvador;2500
f_lix_leclerc;2501
daniel_lavoie;2502
gerhard_sch_ne;2503
g_lben_ergen;2504
georg_kreisler;2505
herbert_gr_nemeyer;2506
herman_van_veen;2507
hildegard_knef;2508
marlene_dietrich;2509
iu;2510
jos_luis_rodr_guez;2511
juliette_gr_co;2512
klaus_hoffmann;2513
konstantin_wecker;2514
saltatio_mortis;2515
luigi_tenco;2516
maria_beth_nia;2517
adriana_calcanhotto;2518
marie_lafor_t;2519
marius_m_ller_westernhagen;2520
mina;2521
no_l_coward;2522
pippo_pollina;2523
rita_lee;2524
os_mutantes;2525
rita_pavone;2526
roger_whittaker;2527
al_bano_romina_power;2528
salvatore_adamo;2529
simone;2530
s_rgio_godinho;2531
udo_j_rgens;2532
udo_lindenberg;2533
ulrich_roski;2534
zaz;2535
z_ramalho;2536
fagner;2537
dith_piaf;2538
duelo;2539
espinoza_paz;2540
fidel_rueda;2541
la_firma;2542
la_arrolladora_banda_el_lim_n;2543
voz_de_mando;2544
sergio_vega;2545
fool_s_garden;2546
waltari;2547
of_montreal;2548
pierre_lapointe;2549
rufus_wainwright;2550
loudon_wainwright_iii;2551
sufjan_stevens;2552
machine_gun_kelly;2553
francesco_guccini;2554
le_orme;2555
lucio_dalla;2556
michel_fugain;2557
al_jarreau;2558
carmen_mcrae;2559
javier_sol_s;2560
harry_connick_jr;2561
bap;2562
cradle_of_filth;2563
amorphis;2564
avatar;2565
bathory;2566
behemoth;2567
borknagar;2568
countess;2569
cruachan;2570
darkthrone;2571
hate;2572
destruction;2573
dimmu_borgir;2574
eisregen;2575
enslaved;2576
finntroll;2577
fates_warning;2578
graveworm;2579
impaled_nazarene;2580
sentenced;2581
king_diamond;2582
kreator;2583
lord_belial;2584
marduk;2585
mercyful_fate;2586
stick_to_your_guns;2587
moonspell;2588
as_i_lay_dying;2589
nunslaughter;2590
rotting_christ;2591
samael;2592
sandy_denny;2593
skyforger;2594
sodom;2595
cannibal_corpse;2596
exodus;2597
atreyu;2598
theatres_des_vampires;2599
wizard;2600
transmetal;2601
venom;2602
belphegor;2603
the_crown;2604
moya_brennan;2605
todd_rundgren;2606
clay_walker;2607
andrew_peterson;2608
lynn_anderson;2609
david_crowder_band;2610
pam_tillis;2611
norah_jones;2612
rhonda_vincent;2613
jamey_johnson;2614
plumb;2615
j_j_cale;2616
new_riders_of_the_purple_sage;2617
joe_diffie;2618
kasey_chambers;2619
leon_russell;2620
jack_greene;2621
the_string_cheese_incident;2622
ystein_sunde;2623
stephen_stills;2624
cancerslug;2625
robert_plant;2626
alvin_lee;2627
beth_hart;2628
jimmy_buffett;2629
billy_s_band;2630
bunbury;2631
nacho_vegas;2632
calogero;2633
georges_brassens;2634
canned_heat;2635
charlie_louvin;2636
colin_james;2637
cuby_blizzards;2638
dick_annegarn;2639
edoardo_bennato;2640
eva_cassidy;2641
gil_scott_heron;2642
glenn_hughes;2643
deep_purple;2644
connie_smith;2645
iva_zanicchi;2646
izzy_stradlin;2647
j_karjalainen;2648
jack_bruce;2649
leonard_cohen;2650
joan_armatrading;2651
joan_osborne;2652
john_martyn;2653
rio_reiser;2654
larry_carlton;2655
madeleine_peyroux;2656
bruce_cockburn;2657
kate_anna_mcgarrigle;2658
mavis_staples;2659
noa;2660
ralph_mctell;2661
renato_carosone;2662
richie_kotzen;2663
robben_ford;2664
roberto_carlos;2665
erasmo_carlos;2666
robin_trower;2667
rory_block;2668
roy_buchanan;2669
sandra_mihanovich;2670
savoy_brown;2671
shirley_horn;2672
siniestro_total;2673
slank;2674
the_fabulous_thunderbirds;2675
the_seatbelts;2676
the_tragically_hip;2677
mike_jones;2678
trophy_scars;2679
caravan;2680
velhas_virgens;2681
walter_trout;2682
gov_t_mule;2683
bar_o_vermelho;2684
blue_cheer;2685
ian_hunter;2686
david_leb_n;2687
de_palmas;2688
eugenio_finardi;2689
extreme;2690
foghat;2691
george_thorogood_the_destroyers;2692
great_white;2693
guardian;2694
jethro_tull;2695
ian_anderson;2696
david_knopfler;2697
steppenwolf;2698
dave_edmunds;2699
lynyrd_skynyrd;2700
crosby_stills_nash;2701
raul_seixas;2702
the_poodles;2703
musiq_soulchild;2704
shocking_blue;2705
nick_lowe;2706
the_black_crowes;2707
traffic;2708
widespread_panic;2709
co;2710
alberto_cortez;2711
joan_sebastian;2712
ana_gabriel;2713
gilberto_santa_rosa;2714
rub_n_blades;2715
v_ctor_manuelle;2716
celia_cruz;2717
luis_fonsi;2718
nek;2719
dr_feelgood;2720
astrud_gilberto;2721
benito_di_paula;2722
brazzaville;2723
sacha_distel;2724
chico_buarque;2725
elis_regina;2726
milton_nascimento;2727
faf_de_bel_m;2728
nikka_costa;2729
tim_maia;2730
gilberto_gil;2731
lisa_ekdahl;2732
joyce;2733
maria_rita;2734
nara_le_o;2735
nouvelle_vague;2736
paulinho_moska;2737
wilson_simonal;2738
14_bis;2739
arnaldo_antunes;2740
biquini_cavad_o;2741
cidade_negra;2742
cpm_22;2743
c_ssia_eller;2744
os_paralamas_do_sucesso;2745
guilherme_arantes;2746
ira;2747
lob_o;2748
nenhum_de_n_s;2749
djavan;2750
rog_rio_skylab;2751
roupa_nova;2752
ultraje_a_rigor;2753
kj_52;2754
amado_batista;2755
chit_ozinho_xoror;2756
jo_o_paulo_daniel;2757
leandro_leonardo;2758
leonardo;2759
odair_jos;2760
kaiser_chiefs;2761
kula_shaker;2762
lightning_seeds;2763
pulp;2764
the_proclaimers;2765
dying_fetus;2766
napalm_death;2767
nile;2768
pathology;2769
hilary_duff;2770
badly_drawn_boy;2771
federico_salvatore;2772
i_gufi;2773
zachary_richard;2774
stan_rogers;2775
moxy_fr_vous;2776
poco;2777
la_bottine_souriante;2778
stompin_tom_connors;2779
bersuit_vergarabat;2780
las_pastillas_del_abuelo;2781
george_lam;2782
altan;2783
clannad;2784
blackmore_s_night;2785
capercaillie;2786
celtic_thunder;2787
eluveitie;2788
powerwolf;2789
gaelic_storm;2790
an_na;2791
jon_anderson;2792
the_dubliners;2793
loreena_mckennitt;2794
omnia;2795
secret_garden;2796
shaun_davey;2797
roger_daltrey;2798
the_corrs;2799
los_tigres_del_norte;2800
laurent_voulzy;2801
the_kelly_family;2802
wolfe_tones;2803
alan_stivell;2804
heather_alexander;2805
kate_rusby;2806
dropkick_murphys;2807
great_big_sea;2808
fiddler_s_green;2809
heather_dale;2810
runrig;2811
the_waterboys;2812
dougie_maclean;2813
adriano_celentano;2814
alain_chamfort;2815
zazie;2816
hamelen;2817
tazenda;2818
arno;2819
arthur_h;2820
boudewijn_de_groot;2821
charles_trenet;2822
claudio_baglioni;2823
claudio_rocchi;2824
fabrizio_de_andr;2825
dalida;2826
dana_winner;2827
demis_roussos;2828
esther_ofarim;2829
eugenio_bennato;2830
michel_berger;2831
francis_cabrel;2832
maxime_le_forestier;2833
georges_moustaki;2834
gianmaria_testa;2835
gianni_morandi;2836
gigliola_cinquetti;2837
milva;2838
gilbert_b_caud;2839
ginette_reno;2840
giuni_russo;2841
guy_b_art;2842
helena_vondr_kov;2843
hugues_aufray;2844
ivan_graziani;2845
ivano_fossati;2846
jacques_bertin;2847
jean_ferrat;2848
juliane_werding;2849
julien_clerc;2850
los_temerarios;2851
katerine;2852
leny_escudero;2853
mathieu_chedid;2854
luca_barbarossa;2855
l_o_ferr;2856
rosenstolz;2857
marc_lavoine;2858
massimo_bubola;2859
mecano;2860
mia_martini;2861
michel_jonasz;2862
michele_zarrillo;2863
fiorello;2864
nada;2865
mercedes_sosa;2866
nino_d_angelo;2867
patrick_bruel;2868
patty_pravo;2869
pierre_bachelet;2870
rainald_grebe;2871
rapha_l;2872
raphael;2873
richard_anthony;2874
roberto_murolo;2875
ron;2876
stefano_rosso;2877
stephan_eicher;2878
vasco_rossi;2879
yves_duteil;2880
yves_jamait;2881
ang_lica;2882
aaron_carter;2883
barry_louis_polisar;2884
yuri;2885
cri_cri;2886
hevisaurus;2887
juice_leskinen;2888
kidz_bop;2889
mara_maravilha;2890
destroyer;2891
scorpions;2892
obk;2893
duncan_dhu;2894
parry_gripp;2895
sandy_junior;2896
the_verve_pipe;2897
the_verve;2898
the_wiggles;2899
veggietales;2900
newsboys;2901
steven_curtis_chapman;2902
toro_y_moi;2903
medi_val_b_bes;2904
aaron_neville;2905
bethel_music;2906
apologetix;2907
gaither_vocal_band;2908
building_429;2909
chris_tomlin;2910
matt_maher;2911
jerusalem;2912
david_meece;2913
debby_boone;2914
elevation_worship;2915
matt_redman;2916
planetshakers;2917
majesty;2918
jump5;2919
lecrae;2920
michael_w_smith;2921
bride;2922
natalie_grant;2923
the_lads;2924
audio_adrenaline;2925
paul_wilbur;2926
psalmen_voor_nu;2927
sawyer_brown;2928
shane_shane;2929
the_echoing_green;2930
twila_paris;2931
watch_tower_bible_and_tract_society;2932
da_t_r_u_t_h;2933
dc_talk;2934
flame;2935
grits;2936
trip_lee;2937
crystal_lewis;2938
the_cross_movement;2939
tobymac;2940
vico_c;2941
mormon_tabernacle_choir;2942
august_burns_red;2943
black_veil_brides;2944
deliverance;2945
opeth;2946
die_happy;2947
disciple;2948
galactic_cowboys;2949
haste_the_day;2950
living_sacrifice;2951
mastodon;2952
mortification;2953
showbread;2954
labyrinth;2955
stryper;2956
the_devil_wears_prada;2957
underoath;2958
whitecross;2959
petra;2960
huntingtons;2961
mxpx;2962
d_a_d;2963
caedmon_s_call;2964
david_and_the_giants;2965
degarmo_and_key;2966
delirious;2967
don_francisco;2968
five_iron_frenzy;2969
geoff_moore;2970
hawk_nelson;2971
grave;2972
larry_norman;2973
randy_stonehill;2974
monty_python;2975
oomph;2976
oficina_g3;2977
white_heart;2978
rescate;2979
rick_wakeman;2980
la_oreja_de_van_gogh;2981
sanctus_real;2982
fun_people;2983
thousand_foot_krutch;2984
tim_hughes;2985
the_o_c_supertones;2986
4him;2987
billy_gilman;2988
aimee_mann;2989
katharine_mcphee;2990
eros_ramazzotti;2991
z_ro;2992
babbie_mason;2993
bebo_norman;2994
judy_garland;2995
carman;2996
cece_winans;2997
trick_daddy;2998
chris_isaak;2999
cocteau_twins;3000
edyta_g_rniak;3001
enrico_ruggeri;3002
ffh;3003
hanson;3004
hawksley_workman;3005
indigo_girls;3006
irene_grandi;3007
jackie_evancho;3008
joy_electric;3009
kelly_price;3010
mary_mary;3011
israel_houghton;3012
phil_wickham;3013
phillips_craig_dean;3014
roch_voisine;3015
rupaul;3016
gregorian;3017
sarah_connor;3018
sugarland;3019
sweetbox;3020
tarja;3021
the_brian_setzer_orchestra;3022
brian_setzer;3023
badfinger;3024
the_moffatts;3025
the_vandals;3026
trans_siberian_orchestra;3027
roy_drusky;3028
burton_cummings;3029
procol_harum;3030
renaissance;3031
the_pretty_things;3032
twisted_sister;3033
bj_rn_eidsv_g;3034
corvus_corax;3035
schelmish;3036
emilie_autumn;3037
epica;3038
katherine_jenkins;3039
scala_kolacny_brothers;3040
take_6;3041
the_roches;3042
tony_banks;3043
to_e_proeski;3044
lacrimosa;3045
16_volt;3046
bj_rn_rosenstr_m;3047
bob_rivers;3048
cledus_t_judd;3049
frankjavcee;3050
george_formby;3051
ninja_sex_party;3052
paul_and_storm;3053
the_arrogant_worms;3054
tripod;3055
el_cuarteto_de_nos;3056
gwar;3057
knorkator;3058
psychostick;3059
rodgau_monotones;3060
los_palominos;3061
charlie_peacock;3062
jesus_culture;3063
michael_card;3064
tenth_avenue_north;3065
carrie_newcomer;3066
nick_drake;3067
aaron_watson;3068
billy_joe_royal;3069
billy_joe_shaver;3070
charlie_landsborough;3071
chris_ledoux;3072
collin_raye;3073
dan_seals;3074
dave_dudley;3075
hellbillies;3076
ed_bruce;3077
emilio_navaira;3078
jean_shepard;3079
freddie_hart;3080
gary_stewart;3081
gene_watson;3082
gian_giovani;3083
gilberto_gilmar;3084
jason_mraz;3085
ilse_delange;3086
john_prine;3087
jake_owen;3088
wynn_stewart;3089
jim_ed_brown;3090
joe_ely;3091
kid_rock;3092
la_toya_jackson;3093
lit;3094
lita_ford;3095
me_first_and_the_gimme_gimmes;3096
lagwagon;3097
melanie;3098
mickey_newbury;3099
paul_brunelle;3100
paula_fernandes;3101
zez_di_camargo_luciano;3102
randy_rogers_band;3103
reverend_horton_heat;3104
rick_renner;3105
rionegro_solim_es;3106
shooter_jennings;3107
terri_clark;3108
vern_gosdin;3109
webb_wilder;3110
ween;3111
38_special;3112
the_beau_brummels;3113
matanza;3114
clawfinger;3115
acid_drinkers;3116
agnostic_front;3117
biohazard;3118
body_count;3119
d_r_i;3120
municipal_waste;3121
neurosis;3122
nuclear_assault;3123
soziedad_alkoholika;3124
suicidal_tendencies;3125
paragon;3126
mario;3127
inna;3128
belinda;3129
bronco;3130
grupo_bryndis;3131
david_bisbal;3132
ram_n_ayala;3133
grant_lee_phillips;3134
the_veronicas;3135
amr_diab;3136
atb;3137
basshunter;3138
dream_theater;3139
frankie_j;3140
baby_bash;3141
sophie_ellis_bextor;3142
grace_jones;3143
laveerre;3144
silkk_the_shocker;3145
parov_stelar;3146
raffaella_carr;3147
elephant_man;3148
saint_etienne;3149
samantha_fox;3150
selena;3151
super_junior;3152
t_a_t_u;3153
tarkan;3154
judie_tzuke;3155
el_kel_iset;3156
yello;3157
franz_ferdinand;3158
chenoa;3159
lucero;3160
tokio;3161
puffy_amiyumi;3162
wink;3163
obie_trice;3164
mystikal;3165
current_93;3166
dark_sanctuary;3167
rome;3168
lord_of_the_lost;3169
bella_morte;3170
mantus;3171
blutengel;3172
clan_of_xymox;3173
dead_can_dance;3174
death_in_june;3175
diary_of_dreams;3176
diorama;3177
helium_vola;3178
illuminate;3179
l_me_immortelle;3180
lacrimas_profundere;3181
killing_joke;3182
m_nchener_freiheit;3183
otto_dix;3184
project_pitchfork;3185
qntal;3186
sopor_aeternus;3187
the_cr_xshadows;3188
unheilig;3189
welle;3190
yendri;3191
carcass;3192
asphyx;3193
bolt_thrower;3194
darkseed;3195
paradise_lost;3196
tiamat;3197
the_damned;3198
pantera;3199
the_amity_affliction;3200
judas_priest;3201
amon_amarth;3202
alesana;3203
atrocity;3204
autopsy;3205
avulsed;3206
sabaton;3207
misfits;3208
iron_fire;3209
centinex;3210
dagoba;3211
dark_tranquillity;3212
asia;3213
deicide;3214
dethklok;3215
dew_scented;3216
edge_of_sanity;3217
escape_the_fate;3218
heaven_shall_burn;3219
hypocrisy;3220
incantation;3221
jungle_rot;3222
kataklysm;3223
krisiun;3224
macabre;3225
malevolent_creation;3226
meshuggah;3227
misanthrope;3228
morbid_angel;3229
dead_kennedys;3230
necro;3231
pig_destroyer;3232
shadows_fall;3233
sinister;3234
six_feet_under;3235
dream_evil;3236
soulfly;3237
the_black_dahlia_murder;3238
between_the_buried_and_me;3239
therion;3240
vader;3241
whitechapel;3242
attila;3243
emmure;3244
miss_may_i;3245
the_acacia_strain;3246
betontod;3247
broilers;3248
dritte_wahl;3249
ohl;3250
slime;3251
terrorgruppe;3252
b_hse_onkelz;3253
frei_wild;3254
k_rbholz;3255
asp;3256
tokio_hotel;3257
queensr_che;3258
amanda_miguel;3259
arabesque;3260
bad_boys_blue;3261
boyce_avenue;3262
parliament;3263
wu_tang_clan;3264
neoton_fam_lia;3265
teena_marie;3266
bobby_womack;3267
agoraphobic_nosebleed;3268
candlemass;3269
electric_wizard;3270
black_sabbath;3271
theatre_of_tragedy;3272
type_o_negative;3273
marie_fredriksson;3274
luna;3275
marissa_nadler;3276
yo_la_tengo;3277
celldweller;3278
hitomi;3279
big_d_and_the_kids_table;3280
alacranes_musical;3281
k_paz_de_la_sierra;3282
assemblage_23;3283
covenant;3284
die_krupps;3285
kodak_black;3286
front_242;3287
haujobb;3288
in_strict_confidence;3289
le_ther_strip;3290
snog;3291
the_darkness;3292
tanzwut;3293
terminal_choice;3294
velvet_acid_christ;3295
vnv_nation;3296
wumpscut;3297
x_fusion;3298
umbra_et_imago;3299
de_vision;3300
deichkind;3301
eisbrecher;3302
herbie_hancock;3303
ana_moura;3304
macaco;3305
skinny_puppy;3306
ayreon;3307
black_moth_super_rainbow;3308
erykah_badu;3309
cocorosie;3310
de_jeugd_van_tegenwoordig;3311
dj_shadow;3312
e_nomine;3313
kmfdm;3314
flying_lotus;3315
goldfrapp;3316
hanzel_und_gretyl;3317
information_society;3318
mc_frontalot;3319
kraftwerk;3320
ladytron;3321
lamb;3322
milk_inc;3323
mind_in_a_box;3324
ministry;3325
m_m;3326
m_nia;3327
pig;3328
pitchshifter;3329
lil_boosie;3330
master_p;3331
mindless_self_indulgence;3332
buzzcocks;3333
vanilla_ice;3334
milie_simon;3335
gianna_nannini;3336
pinback;3337
the_birthday_massacre;3338
archive;3339
99_posse;3340
bloc_party;3341
morcheeba;3342
origa;3343
paul_kalkbrenner;3344
tina_arena;3345
dover;3346
melotron;3347
owl_city;3348
kamelot;3349
greeley_estates;3350
hawthorne_heights;3351
joan_of_arc;3352
saves_the_day;3353
thursday;3354
transit;3355
fairport_convention;3356
maggie_reilly;3357
joan_manuel_serrat;3358
e_rotic;3359
the_scene;3360
sandra;3361
amon_d_l_ii;3362
circa_survive;3363
love_solfege;3364
caliban;3365
tall_dwarfs;3366
van_der_graaf_generator;3367
death_grips;3368
the_fiery_furnaces;3369
am_lia_rodrigues;3370
cristina_branco;3371
jos_afonso;3372
katia_guerreiro;3373
ney_matogrosso;3374
madredeus;3375
mariza;3376
gipsy_kings;3377
mal;3378
aleks_syntek;3379
ni_a_pastori;3380
rosario;3381
al_stewart;3382
amos_lee;3383
andr_s_calamaro;3384
ane_brun;3385
asa;3386
editors;3387
catie_curtis;3388
chrystian_ralf;3389
clueso;3390
eddi_reader;3391
eddie_from_ohio;3392
ellis_paul;3393
frank_turner;3394
estampie;3395
ferdi_tayfur;3396
fito_p_ez;3397
luis_alberto_spinetta;3398
gabriella_ferri;3399
gigi;3400
greg_brown;3401
g_ksel;3402
lando_fiorini;3403
india_arie;3404
jack_savoretti;3405
anne_grete_preus;3406
jarom_r_nohavica;3407
joe_purdy;3408
john_wesley_harding;3409
josh_rouse;3410
karel_kryl;3411
v_tor_ramil;3412
lars_winnerb_ck;3413
laura_marling;3414
llu_s_llach;3415
los_chalchaleros;3416
luka_bloom;3417
malicorne;3418
mark_heard;3419
martin_carthy;3420
nic_jones;3421
le_n_gieco;3422
mijares;3423
nuova_compagnia_di_canto_popolare;3424
ola_magnell;3425
thin_lizzy;3426
ray_lamontagne;3427
ron_sexsmith;3428
rosana;3429
silvio_rodr_guez;3430
stef_bos;3431
sun_kil_moon;3432
tanita_tikaram;3433
the_incredible_string_band;3434
thea_gilmore;3435
tina_dico;3436
victor_leo;3437
v_rttin;3438
ge_aleksandersen;3439
i_brahim_tatl_ses;3440
ektomorf;3441
elvenking;3442
ensiferum;3443
falconer;3444
feuerschwanz;3445
in_extremo;3446
korpiklaani;3447
leaves_eyes;3448
letzte_instanz;3449
m_go_de_oz;3450
saurom;3451
schandmaul;3452
skyclad;3453
subway_to_sally;3454
suidakra;3455
t_r;3456
icehouse;3457
bomb_the_music_industry;3458
the_real_mckenzies;3459
54_40;3460
armored_saint;3461
alexz_johnson;3462
bar_man_o;3463
ezginin_g_nl;3464
galija;3465
sts;3466
h_kan_hellstr_m;3467
james_blunt;3468
kazik;3469
mewithoutyou;3470
michel_polnareff;3471
ovidi_montllor;3472
rasputina;3473
shearwater;3474
gerry_rafferty;3475
steam_powered_giraffe;3476
the_saw_doctors;3477
ty_segall;3478
tyrone_wells;3479
avi_es_do_forr;3480
grimskunk;3481
sinik;3482
vitaa;3483
kenza_farah;3484
sexion_d_assaut;3485
aliz_e;3486
henri_tachan;3487
jenifer;3488
m_pokora;3489
indochine;3490
brainstorm;3491
con_funk_shun;3492
funkadelic;3493
lena_park;3494
neffa;3495
ugk;3496
suburban_legends;3497
mai_kuraki;3498
cherry_poppin_daddies;3499
electric_six;3500
los_straitjackets;3501
the_69_eyes;3502
the_angels;3503
the_haunted;3504
the_hellacopters;3505
the_kills;3506
thee_oh_sees;3507
white_denim;3508
zabranjeno_pu_enje;3509
ol_dirty_bastard;3510
kurupt;3511
spice_1;3512
brotha_lynch_hung;3513
chamillionaire;3514
paul_wall;3515
trae;3516
club_dogo;3517
mc_eiht;3518
royce_da_5_9;3519
geto_boys;3520
the_diplomats;3521
ice_t;3522
2_live_crew;3523
xv;3524
mobb_deep;3525
c_murder;3526
tru;3527
lil_keke;3528
project_pat;3529
tha_dogg_pound;3530
esham;3531
twiztid;3532
erick_sermon;3533
big_tymers;3534
kate_nash;3535
the_cramps;3536
nekromantix;3537
tsol;3538
ace_frehley;3539
hardcore_superstar;3540
harem_scarem;3541
house_of_lords;3542
kingdom_come;3543
l_a_guns;3544
mr_big;3545
pink_cream_69;3546
quiet_riot;3547
riot;3548
ratt;3549
tnt;3550
backyard_babies;3551
ultima_thule;3552
europe;3553
hanoi_rocks;3554
mott_the_hoople;3555
smokie;3556
suzi_quatro;3557
haemorrhage;3558
aline_barros;3559
bruna_karla;3560
kirk_franklin;3561
minist_rio_koinonya_de_louvor;3562
artrosis;3563
closterkeller;3564
indica;3565
sirenia;3566
trail_of_tears;3567
tristania;3568
within_temptation;3569
bauhaus;3570
mono_inc;3571
pansy_division;3572
xandria;3573
immortal_technique;3574
agathocles;3575
rotten_sound;3576
the_locust;3577
anthrax;3578
devildriver;3579
lamb_of_god;3580
machine_head;3581
parkway_drive;3582
pro_pain;3583
throwdown;3584
vicious_rumors;3585
screaming_trees;3586
cuisillos;3587
intocable;3588
pesado;3589
la_mafia;3590
marco_antonio_sol_s;3591
los_bukis;3592
andrew_w_k;3593
april_wine;3594
axel_rudi_pell;3595
b_z;3596
tak_matsumoto;3597
barricada;3598
bijelo_dugme;3599
blaze_bayley;3600
bonfire;3601
bruce_dickinson;3602
buckcherry;3603
budgie;3604
buitres;3605
jorn;3606
doro;3607
enuff_z_nuff;3608
gentle_giant;3609
girlschool;3610
golden_earring;3611
gotthard;3612
nazareth;3613
a_day_to_remember;3614
jefferson_airplane;3615
joe_satriani;3616
ken_hensley;3617
kim_mitchell;3618
king_s_x;3619
kotiteollisuus;3620
la_renga;3621
lee_aaron;3622
lordi;3623
michael_schenker_group;3624
mustasch;3625
night_ranger;3626
omega;3627
parni_valjak;3628
paul_gilbert;3629
popeda;3630
skid_row;3631
tankcsapda;3632
the_bronx;3633
the_donnas;3634
all_that_remains;3635
triumph;3636
umphrey_s_mcgee;3637
y_t;3638
ziggy;3639
sfdk;3640
7_seconds;3641
aiden;3642
alphaville;3643
black_flag;3644
slayer;3645
circle_jerks;3646
ritchie;3647
converge;3648
every_time_i_die;3649
hatebreed;3650
nomeansno;3651
rancid;3652
memphis_may_fire;3653
nofx;3654
propagandhi;3655
tankard;3656
screeching_weasel;3657
sick_of_it_all;3658
silverstein;3659
two_steps_from_hell;3660
faun;3661
accept;3662
the_frames;3663
andromeda;3664
annihilator;3665
anvil;3666
artillery;3667
avenged_sevenfold;3668
axxis;3669
blind_guardian;3670
vanden_plas;3671
grave_digger;3672
dragonforce;3673
edenbridge;3674
damien_jurado;3675
exciter;3676
firewind;3677
halford;3678
hammerfall;3679
helloween;3680
helstar;3681
iced_earth;3682
jag_panzer;3683
machinae_supremacy;3684
manowar;3685
metal_church;3686
morgana_lefay;3687
mudvayne;3688
nocturnal_rites;3689
overkill;3690
primal_fear;3691
rebellion;3692
running_wild;3693
corey_hart;3694
savatage;3695
saxon;3696
steve_vai;3697
tad_morose;3698
tarot;3699
tierra_santa;3700
trivium;3701
turmion_k_til_t;3702
u_d_o;3703
virgin_steele;3704
voivod;3705
warcry;3706
yngwie_malmsteen;3707
zion_lennox;3708
sido;3709
mc_chris;3710
assalti_frontali;3711
kool_keith;3712
ayumi_hamasaki;3713
az;3714
bahh_tee;3715
bassi_maestro;3716
revocation;3717
blumentopf;3718
brockhampton;3719
bts;3720
bushido;3721
vinnie_paz;3722
chakuza;3723
cheek;3724
cro;3725
arc_ngel;3726
alexis_fido;3727
dargen_d_amico;3728
the_coup;3729
def_con_dos;3730
die_fantastischen_vier;3731
dom_no;3732
donguralesko;3733
epmd;3734
kool_savas;3735
fettes_brot;3736
fronda;3737
mc_solaar;3738
pyhimys;3739
kaaris;3740
kollegah;3741
kontra_k;3742
k_k;3743
l_o_c;3744
logic;3745
jerry_rivera;3746
murs;3747
angie_stone;3748
namie_amuro;3749
anthony_hamilton;3750
lyfe_jennings;3751
bl_f;3752
o_s_t_r;3753
paluch;3754
parazi_ii;3755
porta;3756
bleeding_through;3757
prinz_pi;3758
rasmentalism;3759
xavier_naidoo;3760
sage_francis;3761
stupeflip;3762
young_thug;3763
tego_calder_n;3764
fifth_harmony;3765
jay_chou;3766
blitzkid;3767
zumbis_do_espa_o;3768
deer_tick;3769
half_man_half_biscuit;3770
hayden;3771
club_8;3772
grandaddy;3773
jens_lekman;3774
kent;3775
keren_ann;3776
los_campesinos;3777
nellie_mckay;3778
china_crisis;3779
prefab_sprout;3780
the_clientele;3781
the_lucksmiths;3782
bell_x1;3783
british_sea_power;3784
car_seat_headrest;3785
deerhunter;3786
dr_dog;3787
elf_power;3788
frightened_rabbit;3789
fugazi;3790
fury_in_the_slaughterhouse;3791
julie_doiron;3792
tinashe;3793
la_habitaci_n_roja;3794
margot_the_nuclear_so_and_so_s;3795
matt_pond_pa;3796
metric;3797
mike_doughty;3798
mother_mother;3799
piebald;3800
quasi;3801
rheostatics;3802
sebadoh;3803
spoon;3804
starflyer_59;3805
stephen_malkmus;3806
stereolab;3807
ted_leo_and_the_pharmacists;3808
the_appleseed_cast;3809
the_faint;3810
the_go_betweens;3811
the_pineapple_thief;3812
the_undertones;3813
tronic;3814
chris_de_burgh;3815
mass_hysteria;3816
angelo_branduardi;3817
gigi_d_alessio;3818
i_muvrini;3819
back_number;3820
boa;3821
claris;3822
crystal_kay;3823
zard;3824
gackt;3825
garnet_crow;3826
girls_generation;3827
kat_tun;3828
koda_kumi;3829
kotoko;3830
lisa;3831
maaya_sakamoto;3832
masami_okui;3833
mr_children;3834
news;3835
shinee;3836
w_inds;3837
yui;3838
yumi_matsutoya;3839
the_high_lows;3840
sid;3841
abbey_lincoln;3842
anna_maria_jopek;3843
cassandra_wilson;3844
dianne_reeves;3845
fred_buscaglione;3846
jane_monheit;3847
zor_n;3848
kraan;3849
laura_fygi;3850
michael_franks;3851
natalino_otto;3852
quartetto_cetra;3853
scott_bradlee_s_postmodern_jukebox;3854
stacey_kent;3855
the_flower_kings;3856
ronnie_von;3857
brown_eyed_girls;3858
ahmet_kaya;3859
alejandra_guzm_n;3860
ana_carolina;3861
alcione;3862
el_chapo_de_sinaloa;3863
gustavo_cerati;3864
soda_stereo;3865
jenni_rivera;3866
joaqu_n_sabina;3867
los_fabulosos_cadillacs;3868
abel_pintos;3869
ana_bel_n;3870
aterciopelados;3871
camilo_sesto;3872
david_demar_a;3873
gian_marco;3874
menudo;3875
ricardo_arjona;3876
sabroso;3877
v_ctor_manuel;3878
las_pelotas;3879
ariel_pink;3880
leehom_wang;3881
jolin_tsai;3882
darkest_hour;3883
kalmah;3884
nightrage;3885
eppu_normaali;3886
the_outfield;3887
no_use_for_a_name;3888
pennywise;3889
callejon;3890
d_f_c;3891
our_last_night;3892
exaltasamba;3893
beth_carvalho;3894
jo_o_bosco;3895
marina_lima;3896
marisa_monte;3897
nando_reis;3898
natiruts;3899
ra_a_negra;3900
s_pra_contrariar;3901
zeca_pagodinho;3902
andr_hazes;3903
de_dijk;3904
arena;3905
iq;3906
sol_invictus;3907
new_found_glory;3908
adam_ant;3909
berlin;3910
hoodoo_gurus;3911
ultravox;3912
nik_kershaw;3913
squeeze;3914
the_aquabats;3915
the_fixx;3916
beat_crusaders;3917
cows;3918
conjunto_primavera;3919
peter_and_the_test_tube_babies;3920
sham_69;3921
the_adicts;3922
the_analogs;3923
instalok;3924
jacek_kaczmarski;3925
przemys_aw_gintrowski;3926
ada_band;3927
agnetha_f_ltskog;3928
ajda_pekkan;3929
al_bano;3930
alex_ubago;3931
alison_moyet;3932
alunni_del_sole;3933
anna_oxa;3934
bajm;3935
barclay_james_harvest;3936
blue_system;3937
brunner_brunner;3938
candan_er_etin;3939
christian_bautista;3940
clay_aiken;3941
clifford_t_ward;3942
daniel;3943
don_backy;3944
jesse_mccartney;3945
emma;3946
marcella_bella;3947
giorgio_gaber;3948
guus_meeuwis;3949
heinz_rudolf_kunze;3950
john_farnham;3951
ian_thomas;3952
i_n_karaca;3953
jennifer_rush;3954
jo_vally;3955
john_fogerty;3956
julian_lennon;3957
k3;3958
kid_abelha;3959
labv_l_gais_tips;3960
l_vi;3961
lea_salonga;3962
les_wampas;3963
magnus_uggla;3964
mango;3965
maria_mena;3966
massimo_ranieri;3967
max_gazz;3968
michael_learns_to_rock;3969
mietta;3970
mustafa_sandal;3971
nil_fer;3972
peter_frampton;3973
pr_ta_v_tra;3974
pur;3975
rettore;3976
ricchi_e_poveri;3977
rob_de_nijs;3978
sara_bareilles;3979
sasha;3980
sertab_erener;3981
sezen_aksu;3982
stadio;3983
stephen_sondheim;3984
tamara;3985
team_starkid;3986
toto_cutugno;3987
umberto_tozzi;3988
herman_brood;3989
wanessa;3990
zen_caf;3991
bonanza_banzai;3992
bodyjar;3993
bracket;3994
frenzal_rhomb;3995
goldfinger;3996
the_wonder_years;3997
useless_id;3998
camel;3999
hombres_g;4000
leo_jaime;4001
neal_morse;4002
spock_s_beard;4003
new_trolls;4004
opus;4005
piersi;4006
premiata_forneria_marconi;4007
superbus;4008
zmelkoow;4009
boysetsfire;4010
hot_water_music;4011
new_model_army;4012
the_monochrome_set;4013
big_big_train;4014
avantasia;4015
dark_moor;4016
dreamtale;4017
freedom_call;4018
mystic_prophecy;4019
nightmare;4020
rhapsody_of_fire;4021
royal_hunt;4022
sonata_arctica;4023
stratovarius;4024
symphony_x;4025
vision_divine;4026
the_wildhearts;4027
armia;4028
evergrey;4029
lana_lane;4030
nektar;4031
pain_of_salvation;4032
riverside;4033
beardfish;4034
echolyn;4035
eloy;4036
john_wetton;4037
medina_azahara;4038
mostly_autumn;4039
pendragon;4040
rafo_r_ez;4041
the_meteors;4042
against_me;4043
anti_flag;4044
banda_bassotti;4045
cadena_perpetua;4046
descendents;4047
distemper;4048
dogwood;4049
el_ltimo_ke_zierre;4050
farben_lehre;4051
toy_dolls;4052
junkies;4053
ksu;4054
la_polla_records;4055
la_vela_puerca;4056
leatherface;4057
less_than_jake;4058
mad_caddies;4059
millencolin;4060
punkreas;4061
reel_big_fish;4062
snfu;4063
stiff_little_fingers;4064
swingin_utters;4065
the_bouncing_souls;4066
the_casualties;4067
the_dickies;4068
the_lawrence_arms;4069
toyah;4070
gerald_levert;4071
gondwana;4072
los_aut_nticos_decadentes;4073
los_cafres;4074
los_pericos;4075
tryo;4076
rakim_ken_y;4077
billy_squier;4078
bj_rn_afzelius;4079
glay;4080
hunters_collectors;4081
john_entwistle;4082
jokke;4083
la_beriso;4084
los_rancheros;4085
los_tres;4086
maanam;4087
mikel_erentxun;4088
peter_wolf;4089
racoon;4090
rev_lver;4091
riblja_orba;4092
sandro;4093
gene_vincent;4094
the_baseballs;4095
stray_cats;4096
as_marcianas;4097
bruno_marrone;4098
cristiano_ara_jo;4099
fernando_sorocaba;4100
joint_venture;4101
serge_reggiani;4102
ska_p;4103
the_mighty_mighty_bosstones;4104
fu_manchu;4105
jay_jay_johanson;4106
psyche;4107
carlos_gardel;4108
'''
]

In [ ]:
#@title **PLAY PROMPTS**
print('db42.wav')
display(Audio(f'{PROMPTS_DIR}/db42.wav'))
print('led.wav')
display(Audio(f'{PROMPTS_DIR}/led.wav'))
print('treads.wav')
display(Audio(f'{PROMPTS_DIR}/treads.wav'))
print('yes.wav')
display(Audio(f'{PROMPTS_DIR}/yes.wav'))

# **CREDITS**

Please do not remove this cell if you share or build on this notebook
```
Author: @Jimney (Jamie Steeg)

Credits: This notebook is a combination of other notebooks, ideas and technical support from these awesome people

* @Broccaloo
* @johannezz
* @raekwon
* @SMarioMan
* @WandB
* @Zag
* Discord Jukebox Group

Based on jukebox models and code released by OpenAI

License: Non-commercial use only.

Citation: 
@article{dhariwal2020jukebox,
  title={Jukebox: A Generative Model for Music},
  author={Dhariwal, Prafulla and Jun, Heewoo and Payne, Christine and Kim, Jong Wook and Radford, Alec and Sutskever, Ilya},
  journal={arXiv preprint arXiv:2005.00341},
  year={2020}
}
```